# Import

In [54]:
# Images
from IPython.display import Image
import random as random

# How to Aproach

- #### Handle Ambiguity:
  - Ask the 6 W's: Who, How, Why, What, When, Where.
- #### Define the Core Objects
- #### Analyze Relations between the Objects
- #### Investicate Actions of the Objects

# Design Patterns

### Singleton Class
The Singleton design pattern is a creational pattern that ensures a class has only one instance and provides a global point of access to that instance. In Python, you can implement the Singleton pattern as follows:

In [4]:
class Restaurant:
    _instance = None  # Private class variable to hold the single instance

    def __init__(self):
        # Constructor (protected)
        pass  # You can add initialization logic here

    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            cls._instance = cls()
        return cls._instance

# Example usage:
restaurant1 = Restaurant.get_instance()
restaurant2 = Restaurant.get_instance()

# restaurant1 and restaurant2 will be the same instance
print(restaurant1 is restaurant2)  # Output: True

True


### Factory Method

The Factory Method pattern is a creational pattern that provides an interface for creating objects, but it allows subclasses to choose the class to instantiate. In Python, you can implement the Factory Method pattern as follows:

In [5]:
from enum import Enum

class GameType(Enum):
    POKER = 1
    BLACKJACK = 2

class CardGame:
    @staticmethod
    def create_card_game(game_type):
        if game_type == GameType.POKER:
            return PokerGame()
        elif game_type == GameType.BLACKJACK:
            return BlackJackGame()
        return None

class PokerGame(CardGame):
    def play(self):
        return "Playing Poker"

class BlackJackGame(CardGame):
    def play(self):
        return "Playing Blackjack"

# Example usage:
poker_game = CardGame.create_card_game(GameType.POKER)
blackjack_game = CardGame.create_card_game(GameType.BLACKJACK)

print(poker_game.play())      # Output: Playing Poker
print(blackjack_game.play())  # Output: Playing Blackjack

Playing Poker
Playing Blackjack


### ***Decorators***

#### @staticmethod
**Purpose:** Used to define a method that belongs to a class, not an instance, and does not depend on any instance-specific data. Can be called without creating objects of the class

**When to Use:** For utility methods that don't need access to instance attributes. When you want a method to be associated with a class itself.

Example:

In [6]:
class MathUtils:
    @staticmethod
    def add(x, y):
        return x + y

# You can call a static method on the class itself, no instance needed
result = MathUtils.add(3, 5)
print(result)  # Output: 8

8


#### @classmethod
**Purpose:** Used to define a method that belongs to a class, and it receives the class itself (usually named cls) as its first parameter. It can be used to create alternative constructors or work with class-level data.

**When to Use:** For Factory Methods (creating instances of a class). When you need access to class-level data or methods.

Example:

In [7]:
class MyClass:
    class_variable = 10

    def __init__(self, instance_variable):
        self.instance_variable = instance_variable

    @classmethod
    def create_instance(cls, value):
        return cls(value)  # Create an instance of the class

# You can call a class method on the class itself
obj = MyClass.create_instance(20)
print(obj.instance_variable)  # Output: 20

20


In summary, @staticmethod is used for methods that don't depend on instance-specific data, while @classmethod is used for methods that work with class-level data or provide alternative ways to create instances. Both allow you to associate methods with a class rather than instances.






# Exercises

#### Exercise 7.1
**Deck of Cards:** Design the data structures for a generic deck of cards. Explain how you would subclass the data structures to implement blackjack.

**Hints:**
- #153: Note that a "card deck" is very broad. You might want to think about a reasonable scope to the problem.
- #275: How, if at all, will you handle aces?

In [260]:
from enum import Enum
import random

class Suit(Enum):
    CLUB = 0
    DIAMOND = 1
    HEART = 2
    SPADE = 3
    

class Card:
    def __init__(self, face_value, suit):
        self.face_value = face_value
        self.suit = suit
        self.available = True

    def value(self):
        pass

    def is_available(self):
        return self.available

    def mark_unavailable(self):
        self.available = False

    def mark_available(self):
        self.available = True

    def __str__(self):
        # Convert face value to a human-readable rank
        ranks = {1: 'Ace', 11: 'Jack', 12: 'Queen', 13: 'King'}
        rank_str = ranks.get(self.face_value, str(self.face_value))
        return f"{rank_str} of {self.suit}"


class Deck:
    def __init__(self):
        self.cards = []
        for suit in Suit:
            for face_value in range(1,14):
                self.cards.append(BlackJackCard(face_value, suit))
        self.dealt_index = 0

    def set_deck_of_cards(self, deck_of_cards):
        self.cards = deck_of_cards

    def shuffle(self):
        random.shuffle(self.cards)

    def remaining_cards(self):
        return len(self.cards) - self.dealt_index

    def deal_hand(self, number):
        hand = []
        for _ in range(number):
            card = self.deal_card()
            if card:
                hand.append(card)
        return hand

    def deal_card(self):
        if self.dealt_index < len(self.cards):
            card = self.cards[self.dealt_index]
            self.dealt_index += 1
            return card
        return None

class Hand:
    def __init__(self):
        self.cards = []

    def score(self):
        score = 0
        for card in self.cards:
            score += card.value()
        return score

    def add_card(self, card):
        self.cards.append(card)

class BlackJackCard(Card):
    def value(self):
        if self.is_ace():
            return 1
        elif self.face_value >= 11 and self.face_value <= 13:
            return 10
        else:
            return self.face_value

    def is_ace(self):
        return self.face_value == 1

    def min_value(self):
        return 1 if self.is_ace() else self.value()

    def max_value(self):
        return 11 if self.is_ace() else self.value()

class BlackJackHand(Hand):
    def score(self):
        scores = self.possible_scores()
        max_under = float("-inf")
        min_over = float("inf")
        
        for score in scores:
            if score > 21 and score < min_over:
                min_over = score
            elif score <= 21 and score > max_under:
                max_under = score
        
        return min_over if max_under == float("-inf") else max_under

    def possible_scores(self):
        num_aces = 0
        total = 0

        for card in self.cards:
            total += card.value()
            if card.is_ace():
                num_aces += 1

        scores = []
        for _ in range(num_aces + 1):
            scores.append(total)
            if total + 10 <= 21:
                total += 10

        return scores

    def is_busted(self):
        return self.score() > 21

    def is_21(self):
        return self.score() == 21

    def is_blackjack(self):
        return self.is_21() and len(self.cards) == 2


In [261]:
# Import statements (assuming you already have the classes defined)

# Create a deck and shuffle it
deck = Deck()
deck.shuffle()

# Create player and dealer hands
player_hand = BlackJackHand()
dealer_hand = BlackJackHand()

# Deal the initial two cards to the player and dealer
player_hand.add_card(deck.deal_card())
dealer_hand.add_card(deck.deal_card())
player_hand.add_card(deck.deal_card())
dealer_hand.add_card(deck.deal_card())

# Function to display the player's and dealer's hands
def display_hands(player_hand, dealer_hand):
    print("Player's Hand:", ", ".join(str(card) for card in player_hand.cards))
    print("Player's Score:", player_hand.score())
    print("\nDealer's Hand:", ", ".join(str(card) for card in dealer_hand.cards))
    print("Dealer's Score:", dealer_hand.score())

# Function to check if the player wants to hit or stand
def player_choice():
    while True:
        choice = input("Do you want to 'hit' or 'stand'? ").lower()
        if choice in ['hit', 'stand']:
            return choice

# Initial display of hands
display_hands(player_hand, dealer_hand)

# Player's turn
while not player_hand.is_busted() and player_choice() == 'hit':
    player_hand.add_card(deck.deal_card())
    display_hands(player_hand, dealer_hand)

# Dealer's turn
while dealer_hand.score() < 17:
    dealer_hand.add_card(deck.deal_card())

# Display final hands
display_hands(player_hand, dealer_hand)

# Determine the winner
if player_hand.is_busted() or (dealer_hand.score() >= player_hand.score() and not dealer_hand.is_busted()):
    print("Dealer wins!")
elif dealer_hand.is_busted() or player_hand.score() > dealer_hand.score():
    print("Player wins!")
else:
    print("It's a tie!")

Player's Hand: 3 of Suit.DIAMOND, 6 of Suit.CLUB
Player's Score: 9

Dealer's Hand: Ace of Suit.SPADE, Queen of Suit.HEART
Dealer's Score: 21


KeyboardInterrupt: Interrupted by user

#### Exercise 7.2
**Call Center:** Imagine you have a call center with three levels of employees: respondent, manager, and director. An incoming telephone call must be first allocated to a respondent who is free. If the respondent can't handle the call, he or she must escalate the call to a manager. If the manager is not free or not able to handle it, then the call should be escalated to a director. Design the classes and data structures for this problem. Implement a method dispatchCall() which assigns a call to the first available employee.

**Hints:**
- #363: Before coding, make a list of the objects you need and walk through the common algorithms. Picture the code. Do you have everything you need? 

In [244]:
class CallCenter:
    _instance = None # Private class variable to hold the single instance
    _respondents = []
    _managers = []
    _directors = []
    _dbussy = False
    _mbussy = False
    _rbussy = False
    _calls = []
    
    def __init__(self):
        # Constructor (protected)
        pass

    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            cls._instance = cls()
        return cls._instance
        
    @classmethod
    def get_employees(cls):
        return _respondents, _managers, _directos
        
    @classmethod
    def set_employees(cls, rnames, mnames, dnames):
        if dnames != None:
            cls._directors = [Director(dnames[i]) for i in range(len(dnames))]
        if mnames != None:
            cls._managers = [Manager(mnames[i]) for i in range(len(mnames))]
        if rnames != None:
            cls._respondents = [Respondent(rnames[i]) for i in range(len(rnames))]
            
    @classmethod
    def check_ds_bussy(cls):
        for i in cls._directors:
            if i.check_bussy() == False:
                cls._dbussy = False
                return cls._dbussy
        cls._dbussy = True
        return cls._dbussy

    @classmethod
    def check_ms_bussy(cls):
        for i in cls._managers:
            if i.check_bussy() == False:
                cls._mbussy = False
                return cls._mbussy
        cls._mbussy = True
        return cls._mbussy

    @classmethod
    def check_rs_bussy(cls):
        for i in cls._respondents:
            if i.check_bussy() == False:
                cls._rbussy = False
                return cls._rbussy
        cls._rbussy = True
        return cls._rbussy

    @classmethod
    def in_call(cls, call_name):
        cls._calls.append(call_name)

    @classmethod
    def out_call(cls, call_name):
        cls._calls.remove(call_name)

    @classmethod
    def dispatch_call(cls, call_name):
        if cls.check_rs_bussy() == False:
            for r in cls._respondents:
                if r.check_bussy() == False:
                    r.dispatch_call()
                    if call_name in cls._calls:
                        cls._calls.remove(call_name)
                    print(r.who, ' ', r.name, ' attending the incomming call: ', call_name, 'is_bussy = ', r.bussy)
                    break
        
        elif cls.check_ms_bussy() == False:
            for m in cls._managers:
                if m.check_bussy() == False:
                    m.dispatch_call()
                    if call_name in cls._calls:
                        cls._calls.remove(call_name)
                    print(m.who, ' ', m.name, ' attending the incomming call: ', call_name, 'is_bussy = ', m.bussy)
                    break
                    
        elif cls.check_ds_bussy() == False:
            for d in cls._directors:
                if d.check_bussy() == False:
                    d.dispatch_call()
                    if call_name in cls._calls:
                        cls._calls.remove(call_name)
                    print(d.who, ' ', d.name, ' attending the incomming call: ', call_name, 'is_bussy = ', d.bussy)
                    break
                    
        else:
            print('all our agents are bussy, please wait until he can handle this')
            cls.in_call(call_name)

        return


class Employee:
    def __init__(self):
        self.bussy = False
        self.who = None

    def check_bussy(self):
        return self.bussy

    def is_bussy(self):
        self.bussy = True

    def is_free(self):
        self.bussy = False

    def dispatch_call():
        pass

class EmployeeType(Enum):
    RESPONDENT = 0
    MANAGER = 1
    DIRECTOR = 2

class Respondent(Employee):
    def __init__(self, name):
        self.bussy = False
        self.who = EmployeeType['RESPONDENT'].name
        self.name = name

    def dispatch_call(self):
        if self.check_bussy() == False:
            self.is_bussy()
        else:
            print('Already Bussy')
        return
            
class Manager(Employee):
    def __init__(self, name):
        self.bussy = False
        self.who = EmployeeType['MANAGER'].name
        self.name = name

    def dispatch_call(self):
        if self.check_bussy() == False:
            self.is_bussy()
        else:
            print('Already Bussy')
        return

class Director(Employee):
    def __init__(self, name):
        self.bussy = False
        self.who = EmployeeType['DIRECTOR'].name
        self.name = name

    def dispatch_call(self):
        if self.check_bussy() == False:
            self.is_bussy()
        else:
            print('Please Wait until one of our agents is free')
        return
        

In [245]:
# Generate Call Center and Check Singleton
callCenter = CallCenter()
callCenter2 = CallCenter()

callCenter._instance == callCenter2._instance

# Generate Employees List:
rnames = ['j', 'ju', 'jua', 'juan']
mnames = ['a', 'as', 'asi']
dnames = ['y', 'yo']

callCenter.set_employees(rnames, mnames, dnames)

# Print Employees Working
print('Respondants: ')
for r in callCenter2._respondents:
    print(' - Name: ', r.name, ' - Bussy: ', r.bussy, ' - Type: ', r.who)
    
print('Managers: ')
for r in callCenter._managers:
    print(' - Name: ', r.name, ' - Bussy: ', r.bussy, ' - Type: ', r.who)
    
print('Directors: ')
for r in callCenter._directors:
    print(' - Name: ', r.name, ' - Bussy: ', r.bussy, ' - Type: ', r.who)




# In Calls
callCenter.dispatch_call('x')
callCenter.dispatch_call('xx')
callCenter.dispatch_call('xxx')
callCenter.dispatch_call('xxxx')
callCenter.dispatch_call('xxxxx')
callCenter.dispatch_call('xxxxxx')
callCenter.dispatch_call('xxxxxxx')
callCenter.dispatch_call('xxxxxxxx')
callCenter.dispatch_call('xxxxxxxxx')
callCenter.dispatch_call('xxxxxxxxxx')


Respondants: 
 - Name:  j  - Bussy:  False  - Type:  RESPONDENT
 - Name:  ju  - Bussy:  False  - Type:  RESPONDENT
 - Name:  jua  - Bussy:  False  - Type:  RESPONDENT
 - Name:  juan  - Bussy:  False  - Type:  RESPONDENT
Managers: 
 - Name:  a  - Bussy:  False  - Type:  MANAGER
 - Name:  as  - Bussy:  False  - Type:  MANAGER
 - Name:  asi  - Bussy:  False  - Type:  MANAGER
Directors: 
 - Name:  y  - Bussy:  False  - Type:  DIRECTOR
 - Name:  yo  - Bussy:  False  - Type:  DIRECTOR
RESPONDENT   j  attending the incomming call:  x is_bussy =  True
RESPONDENT   ju  attending the incomming call:  xx is_bussy =  True
RESPONDENT   jua  attending the incomming call:  xxx is_bussy =  True
RESPONDENT   juan  attending the incomming call:  xxxx is_bussy =  True
MANAGER   a  attending the incomming call:  xxxxx is_bussy =  True
MANAGER   as  attending the incomming call:  xxxxxx is_bussy =  True
MANAGER   asi  attending the incomming call:  xxxxxxx is_bussy =  True
DIRECTOR   y  attending the incom

----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------- OPTIMIZED ----------------

----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------

In [255]:
from enum import Enum
from queue import Queue

class Rank(Enum):
    RESPONDER = 0
    MANAGER = 1
    DIRECTOR = 2

class Call:
    def __init__(self, caller):
        self.rank = Rank.RESPONDER  # Default rank is Responder
        self.caller = caller
        self.handler = None

    def set_handler(self, employee):
        self.handler = employee
        self.rank = employee.get_rank()

    def reply(self, message):
        print(f"Call from {self.caller} received by {self.handler} with message: {message}")

    def get_rank(self):
        return self.rank

    def set_rank(self, rank):
        self.rank = rank

class Employee:
    def __init__(self, name, call_center):
        self.name = name
        self.call_center = call_center
        self.busy = False
        self.rank = Rank.RESPONDER  # Default rank is Responder

    def receive_call(self, call):
        self.busy = True
        call.set_handler(self)
        print(f"{self} is handling a call from {call.caller}")

    def call_completed(self):
        self.busy = False
        self.call_center.assign_call(self)

    def escalate_and_reassign(self):
        if self.rank == Rank.RESPONDER:
            self.call_center.assign_call(self)
        elif self.rank == Rank.MANAGER:
            self.call_center.assign_call(self)
        # Directors can't escalate further

    def assign_new_call(self):
        return self.call_center.assign_call(self)

    def is_free(self):
        return not self.busy

    def get_rank(self):
        return self.rank

    def __str__(self):
        return f"{self.name} ({self.rank.name})"

class CallCenter:
    def __init__(self):
        self.levels = 3
        self.num_respondents = 10
        self.num_managers = 4
        self.num_directors = 2
        self.employees = {
            Rank.RESPONDER: [],
            Rank.MANAGER: [],
            Rank.DIRECTOR: []
        }
        self.call_queues = {
            Rank.RESPONDER: Queue(),
            Rank.MANAGER: Queue(),
            Rank.DIRECTOR: Queue()
        }
        self.setup_employees()

    def setup_employees(self):
        for i in range(self.num_respondents):
            self.employees[Rank.RESPONDER].append(Employee(f"Responder-{i}", self))
        for i in range(self.num_managers):
            self.employees[Rank.MANAGER].append(Employee(f"Manager-{i}", self))
        for i in range(self.num_directors):
            self.employees[Rank.DIRECTOR].append(Employee(f"Director-{i}", self))

    def get_handler_for_call(self, call):
        for rank in Rank:
            employees = self.employees[rank]
            for employee in employees:
                if employee.is_free():
                    return employee
        return None

    def dispatch_call(self, caller_name):
        caller = f"Caller-{caller_name}"
        call = Call(caller)
        employee = self.get_handler_for_call(call)
        if employee:
            employee.receive_call(call)
        else:
            self.call_queues[Rank.RESPONDER].put(call)
            print(f"All agents are busy. {caller} is waiting in the queue.")

    def assign_call(self, employee):
        employee_rank = employee.get_rank()
        for rank in Rank:
            if rank.value > employee_rank.value and not self.call_queues[rank].empty():
                call = self.call_queues[rank].get()
                employee.receive_call(call)
                return True
        return False

# Use case to test the Call Center
if __name__ == "__main__":
    call_center = CallCenter()

    # Dispatch some calls
    call_center.dispatch_call(1)
    call_center.dispatch_call(2)
    call_center.dispatch_call(3)

    # Simulate employees completing calls
    responders = call_center.employees[Rank.RESPONDER]
    responders[0].call_completed()
    responders[1].call_completed()
    responders[2].call_completed()

    # Dispatch more calls
    call_center.dispatch_call(4)
    call_center.dispatch_call(5)

    # Simulate managers and directors completing calls
    managers = call_center.employees[Rank.MANAGER]
    directors = call_center.employees[Rank.DIRECTOR]
    managers[0].call_completed()
    directors[0].call_completed()

    # Dispatch more calls
    call_center.dispatch_call(6)
    call_center.dispatch_call(7)
    call_center.dispatch_call(8)
    call_center.dispatch_call(9)
    call_center.dispatch_call(10)
    call_center.dispatch_call(11)
    call_center.dispatch_call(12)
    call_center.dispatch_call(13)
    call_center.dispatch_call(14)
    call_center.dispatch_call(15)
    call_center.dispatch_call(16)
    call_center.dispatch_call(17)
    call_center.dispatch_call(18)
    call_center.dispatch_call(19)
    call_center.dispatch_call(20)
    call_center.dispatch_call(21)
    call_center.dispatch_call(22)

Responder-0 (RESPONDER) is handling a call from Caller-1
Responder-1 (RESPONDER) is handling a call from Caller-2
Responder-2 (RESPONDER) is handling a call from Caller-3
Responder-0 (RESPONDER) is handling a call from Caller-4
Responder-1 (RESPONDER) is handling a call from Caller-5
Responder-2 (RESPONDER) is handling a call from Caller-6
Responder-3 (RESPONDER) is handling a call from Caller-7
Responder-4 (RESPONDER) is handling a call from Caller-8
Responder-5 (RESPONDER) is handling a call from Caller-9
Responder-6 (RESPONDER) is handling a call from Caller-10
Responder-7 (RESPONDER) is handling a call from Caller-11
Responder-8 (RESPONDER) is handling a call from Caller-12
Responder-9 (RESPONDER) is handling a call from Caller-13
Manager-0 (RESPONDER) is handling a call from Caller-14
Manager-1 (RESPONDER) is handling a call from Caller-15
Manager-2 (RESPONDER) is handling a call from Caller-16
Manager-3 (RESPONDER) is handling a call from Caller-17
Director-0 (RESPONDER) is handl

#### Exercise 7.3
**Jukebox:** Design a musical jukebox using object oriented principles

**Hints:**
- #198: Scope the problem first and make a list of your assumptions. It's often okay to make reasonable assumptions, but you need to make them explicit. 

In [326]:
from queue import Queue
import time

In [337]:
class Jukebox:
    def __init__(self, tracks):
        self.tracks = tracks
        self.money = 0
        self.queue = Queue()

    def listener_select(self, i):
        print(f"Track {i-1}: ", self.tracks[i-1], f" Track {i}: ", self.tracks[i], f" Track {i+1}: ", self.tracks[i+1])
        choice = input("Which track you want to select?: 'next', 'queue', 'prev', 'done'").lower()
        if choice in ['next', 'queue', 'prev', 'done']:
            return choice
        elif choice in self.tracks:
            return choice
            
    def select_track(self):
        i = 0
        choice = 'null'
        while  choice != 'done':
            choice = self.listener_select(i)
            if choice == 'next':
                i += 1
            elif choice == 'prev':
                i -= 1
            elif choice in self.tracks:
                self.queue.put(choice)
                self.get_queue(1)
            elif choice == 'queue':
                self.queue.put(self.tracks[i])
                self.get_queue(1)

    def get_queue(self, pr = 0):
        if pr == 1:
            temp_queue = Queue()
            while not self.queue.empty():
                item = self.queue.get()
                print(f"Observing item: {item}")
                temp_queue.put(item)

            # Restore the elements to the original queue
            while not temp_queue.empty():
                item = temp_queue.get()
                self.queue.put(item)
                
        if self.queue.empty():
            print("\t There is no queue...")
            
        return self.queue

    def get_tracks(self):
        return self.tracks

    def print_tracks(self):
        for i, t in enumerate(self.tracks):
            print(f"\t {i}.-", t)

    def pay(self):
        cash = int(input('Introduce Nº of Coins: 1, 2, 3...: '))
        self.money += cash

    def playback(self):
        if self.queue.empty():
            print('Nothing to Play')

        else:
            cashout = self.queue.qsize()
            while cashout > self.money:
                choice = self.pay_or_play(cashout)
                if choice == 'play':
                    # Reduce Queue
                    self.reduce_queue(cashout-self.money)
                    break
            # Play
            self.play()
        return

    def play(self):
        songs = 0
        while not self.queue.empty():
            songs += 1
            track = self.queue.get()
            print(f"Track: {track} is being played")
            time.sleep(1)
            print("\t ...", end = "")
            time.sleep(1)
            print("\t ...", end = "")
            time.sleep(1)
            print("\t ...")
            time.sleep(1)
            print("\t ...")

        print('FIN DE LA RETRASMISION')
        if self.money - songs > 0:
            print('NO SE OLVIDE SU CAMBIO ;)')

        elif self.money - songs < 0:
            print('EYO WTF MI LOCO PIDETE UN CHUPITO AL MENOS')

        else:
            print('VUELVA PRONTOO :D')
        return
            
    def pay_or_play(self, cashout):
        choice = 'nana'
        while choice != "pay" or choice != "play":
            choice = input(f"Enough $$$ To Reproduce; 'pay' or 'play' anyway?: {cashout-self.money}").lower()
            if choice == 'play':
                self.queue
            elif choice == 'pay':
                self.pay()
            return choice
                
    def reduce_queue(self, elements_to_remove):
        # Temporary storage for the elements to be removed
        removed_elements = []

        # Dequeue elements to be removed from the end
        while not self.queue.empty() and elements_to_remove > 0:
            removed_element = self.queue.get()
            removed_elements.append(removed_element)
            elements_to_remove -= 1

        # Re-enqueue the removed elements back into the queue
        for element in removed_elements:
            self.queue.put(element)
        print(f"The Following Tracks have been deleted from the queue: {removed_elements}")
        
        return self.queue
        
        

In [339]:
tracks = ['yungbi', 'sunny day', 'colors', 'diamonds', 'if you ever', 'legendary', 'spaceship', 'melanine', 'dopamine', 'G']
jukebox = Jukebox(tracks)

def print_juke(jukebox):
    print('TRAKS: ')
    jukebox.print_tracks()
    print("\nQUEUE") 
    jukebox.get_queue(1)
    print("\nMONEY: ", jukebox.money, '$$')

print_juke(jukebox)

jukebox.select_track()
jukebox.pay()
jukebox.playback()



TRAKS: 
	 0.- yungbi
	 1.- sunny day
	 2.- colors
	 3.- diamonds
	 4.- if you ever
	 5.- legendary
	 6.- spaceship
	 7.- melanine
	 8.- dopamine
	 9.- G

QUEUE
	 There is no queue...

MONEY:  0 $$
Track -1:  G  Track 0:  yungbi  Track 1:  sunny day


Which track you want to select?: 'next', 'queue', 'prev', 'done' queue


Observing item: yungbi
Track -1:  G  Track 0:  yungbi  Track 1:  sunny day


Which track you want to select?: 'next', 'queue', 'prev', 'done' spaceship


Observing item: yungbi
Observing item: spaceship
Track -1:  G  Track 0:  yungbi  Track 1:  sunny day


Which track you want to select?: 'next', 'queue', 'prev', 'done' done
Introduce Nº of Coins: 1, 2, 3...:  2


Track: yungbi is being played
	 ...	 ...	 ...
	 ...
Track: spaceship is being played
	 ...	 ...	 ...
	 ...
FIN DE LA RETRASMISION
VUELVA PRONTOO :D


--------------------------
--------------------------
------------------------
-------------------------------------- JUKEBOX OPTIMIZED -----------------------------------------------
-----------

--------------------------
---------------------------
---------------------------


In [346]:
import time

class Jukebox_opt:
    def __init__(self, tracks):
        self.tracks = tracks
        self.money = 0
        self.queue = []

    def listener_select(self, i):
        print(f"Track {i-1}: {self.tracks[i-1]}  Track {i}: {self.tracks[i]}  Track {i+1}: {self.tracks[i+1]}")
        while True:
            choice = input("Which track do you want to select? ('next', 'queue', 'prev', 'done')").lower()
            if choice in ['next', 'prev', 'done']:
                return choice
            elif choice in self.tracks:
                return choice
            
    def select_track(self):
        i = 0
        choice = 'null'
        while choice != 'done':
            choice = self.listener_select(i)
            if choice == 'next':
                i += 1
            elif choice == 'prev':
                i -= 1
            elif choice in self.tracks:
                self.queue.append(choice)
                self.get_queue()
            elif choice == 'queue':
                self.queue.append(self.tracks[i])
                self.get_queue()

    def get_queue(self):
        if not self.queue:
            print("\t There is no queue...")
        else:
            print("Queue:", self.queue)
            
    def get_tracks(self):
        return self.tracks

    def print_tracks(self):
        for i, t in enumerate(self.tracks):
            print(f"\t {i}.-", t)

    def pay(self):
        cash = int(input('Insert Coins (1, 2, 3...): '))
        self.money += cash

    def playback(self):
        if not self.queue:
            print('Nothing to Play')
        else:
            cashout = len(self.queue)
            while cashout > self.money:
                self.pay()
            self.play()

    def play(self):
        for track in self.queue:
            print(f"Track: {track} is being played")
            time.sleep(1)
            print("\t ...", end="")
            time.sleep(1)
            print("\t ...", end="")
            time.sleep(1)
            print("\t ...")
            time.sleep(1)
            print("\t ...")

        print('END OF PLAYBACK')
        if self.money > 0:
            print("Don't forget your change ;)")

        elif self.money < 0:
            print('Enjoy your music!')

        else:
            print('Come back soon! :D')

tracks = ['yungbi', 'sunny day', 'colors', 'diamonds', 'if you ever', 'legendary', 'spaceship', 'melanine', 'dopamine', 'G']
jukebox = Jukebox_opt(tracks)

def print_juke(jukebox):
    print('TRACKS: ')
    jukebox.print_tracks()
    print("\nQUEUE") 
    jukebox.get_queue()
    print("\nMONEY:", jukebox.money, '$$')

print_juke(jukebox)

jukebox.select_track()
jukebox.pay()
jukebox.playback()

TRACKS: 
	 0.- yungbi
	 1.- sunny day
	 2.- colors
	 3.- diamonds
	 4.- if you ever
	 5.- legendary
	 6.- spaceship
	 7.- melanine
	 8.- dopamine
	 9.- G

QUEUE
	 There is no queue...

MONEY: 0 $$
Track -1: G  Track 0: yungbi  Track 1: sunny day


Which track do you want to select? ('next', 'queue', 'prev', 'done') spaceship


Queue: ['spaceship']
Track -1: G  Track 0: yungbi  Track 1: sunny day


Which track do you want to select? ('next', 'queue', 'prev', 'done') yungbi


Queue: ['spaceship', 'yungbi']
Track -1: G  Track 0: yungbi  Track 1: sunny day


Which track do you want to select? ('next', 'queue', 'prev', 'done') done
Insert Coins (1, 2, 3...):  1
Insert Coins (1, 2, 3...):  22


Track: spaceship is being played
	 ...	 ...	 ...
	 ...
Track: yungbi is being played
	 ...	 ...	 ...
	 ...
END OF PLAYBACK
Don't forget your change ;)


#### Exercise 7.4
**Parking Lot:** Design a parking lot using object-oriented principles.

**Hints:**
- #258: Does the parking lot have multiple levels? What "features" does it support? Is it paid? What types of vehicles?

In [2]:
from enum import Enum
from datetime import datetime, timedelta

In [102]:
class ParkingTipe(Enum):
    GAS = 0
    ELECTRIC = 1

class Fee(Enum):
    DAY = 0
    NOON = 1
    NIGHT = 2

class ParkingArea:
    def __init__(self, id, lvl, tipe, cost):
        if type(tipe) != ParkingTipe or lvl == None:
            raise Exception('Wrong Initialization')
        
        self.id = id 
        self.lvl = lvl
        self.tipe = tipe 
        self.cost = cost
        self.user = None
        self.free = True

    # Check if Free, Is Free, Is Bussy
    def check_if_free(self):
        if self.get_user() == None:
            self.is_free()
        else:
            self.is_bussy()
        return self.free
            
    def is_free(self):
        self.free = True
            
    def is_bussy(self):
        self.free = False

    # User Parks, Get User, Set User
    def user_parks(self, user):
        if self.matching_tipes(user):
            self.set_user(user)
            self.is_bussy()
            user.set_time_in(datetime.now())

        else:
            print(f"You can not park here; This is a {self.tipe.name} station and you are a {user.tipe.name} user")
            return -1

        return 1

    def get_user(self):
        return self.user

    def set_user(self, user):
        self.user = user

    def user_leaves(self):
        self.user.set_time_out(datetime.now())
        self.user.pay()
        self.user.area = None
        self.is_free()
        self.user = None

    # Is User Tipe == Parking Tipe??
    def matching_tipes(self, user):
        if user.tipe != self.tipe:
            return False
        return True

    def get_cost(self):
        return self.cost

    def set_cost(self, cost):
        self.cost = cost

class User:
    def __init__(self, id_plate, tipe, parking_lot):
        self.id = id_plate
        self.tipe = tipe
        self.parking_lot = parking_lot
        self.time_in = datetime.now()
        self.time_out = None
        self.lvl = None
        self.area = None
        for l in parking_lot.get_lvls():
            if l.check_if_free() == True:
                self.lvl = l
                break
        
        if self.lvl == None:
            print('Sorry, at the moment we have no free parking Areas, You have to leave')
            return None
        else:
            for a in self.lvl.get_areas():
                if a.check_if_free() == True and a.tipe == self.tipe:
                    self.area = a
                    self.park()
                    break
        if self.area == None:
            print('Sorry, at the moment we have no free parking Areas, You have to leave')
            return None
        else:
            print(f'WELCOME: {self.id}...\n\t Your current area is AREA: {self.area.id} - LVL: {self.lvl} \n')
            
    # Get in, Go out: Level, Parking Lot and Area:
    def set_parking_lot(self, parking_lot):
        self.parking_lot = parking_lot

    def get_parking_lot(self):
        return self.parking_lot
        
    def set_lvl(self, lvl):
        self.lvl = lvl

    def get_lvl(self):
        return self.lvl

    # Get and Set/(park) Area
    def park(self):
        compatibility = self.get_area().user_parks(self)
        if compatibility == -1:
            return None

    def get_area(self):
        return self.area

    def go_out(self):
        if self.parking_lot != None and self.area != None:
            self.area.user_leaves()

        else:
            print('You cant go out because you are not in an AREA or PARKING LOT')
    
    # TIME
    def get_time_in(self):
        return self.time_in

    def set_time_in(self, time):
        self.time_in = time

    def get_time_out(self):
        return self.time_out

    def set_time_out(self, time):
        self.time_out = time

    def pay(self):
        # The parking Lot is Free
        if self.parking_lot.get_cost() == None:
            print(f'Goodbye, Thanks for your visit.\nHope you come to see you again in {self.get_parking_lot().get_name()} ;)')
            self.set_lvl(None)
            self.get_parking_lot().bye_user(self)
            self.set_parking_lot(None)
            return

        else:
            lvl_cost = self.get_lvl().get_cost()
            area_cost = self.get_area().get_cost()
            parking_lot_hours = self.get_time_out() - self.get_time_in()
            delta_in_minutes = int(parking_lot_hours.total_seconds() / 60)
            cost = ((lvl_cost + area_cost) * delta_in_minutes)
            payed = 0
            choice = 0
            while payed < cost:
                choice = int(input(f"You have to pay a total of {cost}; please insert coins $$:"))
                if type(choice) == int or type(coice) == float:
                    payed += choice

            self.set_lvl(None)
            self.get_parking_lot().bye_user(self)
            print(f'Goodbye, Thanks for your visit.\nHope you come to see you again in {self.get_parking_lot().get_name()}\n\t dont forget your change: {payed-choice} ;)')
            self.set_parking_lot(None)
            return
        

class Level:
    def __init__(self, lvl, n_parkings, n_gas_parkings, cost, g_a_costs, e_a_costs):
        self.lvl = lvl # Name
        self.areas = [ParkingArea(i, lvl, ParkingTipe.GAS, g_a_costs) if i < n_gas_parkings else ParkingArea(i, lvl, ParkingTipe.ELECTRIC, e_a_costs) for i in range(n_parkings)]
        self.free = True
        self.cost = cost

    def check_if_free(self):
        for area in self.areas:
            if area.free == True:
                self.free = True
                return self.free

        self.free = False
        return self.free
    
    def get_areas(self):
        return self.areas
    
    def get_cost(self):
        return self.cost

    def set_cost(self, cost):
        self.cost

    def get_level():
        return self.lvl
        
class ParkingLot:
    def __init__(self, name, n_lvls, n_parkings, n_gas_parkings, l_costs, g_a_costs, e_a_costs, cost):
        self.name = name
        self.lvls = [Level(i, n_parkings, n_gas_parkings, l_costs, g_a_costs, e_a_costs) for i in range(n_lvls)]
        self.users = [] # Max Tam = n_lvls * n_parkings
        self.lvls_free = [] # Max Tam = n_lvls
        self.cost = cost

    def get_name(self):
        return self.name
        
    def get_cost(self):
        return self.cost
        
    def get_lvls(self):
        return self.lvls

    def add_lvl(self, name, n_parkings, n_gas_parkings, l_cost, g_a_costs, e_a_costs):
        self.lvls.append(Level(name, n_parkings, n_gas_parkings, l_cost, g_a_costs, e_a_costs))

    '''def delete_lvl():''' # Se me complica

    def get_users(self):
        return self.users

    def add_users(self, id, tipe):
        user = User(id, tipe, self)
        if user.area != None:
            self.users.append(user)

    def bye_user(self, user):
        if user in self.get_users():
            self.get_users().remove(user)
        else:
            print(f'The user: {user.id}, Is not in the parking lot')
            
    def check_lvls_free(self):
        for lvl in self.lvls:
            if lvl.check_if_free == True:
                self.lvls_free = True
                return self.lvls_free

        self.lvls_free = False
        return lvls_free

    
    

In [103]:
def print_user(user):
    print(f'USER: {user.id} \n\t - Type: {user.tipe} \n\t - time_in: {user.time_in} \n\t - time_out: {user.time_out} \n\t - lvl: {user.lvl} \n\t - area: {user.area.id}')

def print_lvl(lvl):
    print(f'LVL: {lvl.lvl} \n\t - n_areas: {len(lvl.areas)} \n\t - Free: {lvl.check_if_free()} \n\t - costs: {cost}')

def print_area(area):
    print(f'\n\tAREA: {area.id} --> LVL :{area.lvl} \n\t\t - Type: {area.tipe} \n\t\t - Cost: {area.cost} \n\t\t - Free: {area.free} \n\t\t - User: {area.user}')

def print_parking_lot(parking_lot):
    print('___________________________________________________________________________________________________________________________________') 
    print('USERS -----------------------------------------------------------------------------------------------------------------------------') 

    for u in parking_lot.get_users():
        print_user(u)
        
    print('LVLS and AREAS --------------------------------------------------------------------------------------------------------------------') 
    for l in parking_lot.get_lvls():
        print_lvl(l)
        for a in l.get_areas():
            print_area(a)

    print('___________________________________________________________________________________________________________________________________') 


In [104]:
n_lvls = 3
n_parkings = 3
n_gas_parkings = 2

l_cost = 5
g_a_costs = 0
e_a_costs = 5
cost = 10

parking_lot = ParkingLot('AYAWASKA WEY', n_lvls, n_parkings, n_gas_parkings, l_cost, g_a_costs, e_a_costs, cost)
print_parking_lot(parking_lot)
print('\n\n')

# Make Parking Lot Full

parking_lot.add_users('1', ParkingTipe.GAS)
parking_lot
parking_lot.add_users('2', ParkingTipe.GAS)
parking_lot.add_users('X', ParkingTipe.ELECTRIC)

parking_lot.add_users('3', ParkingTipe.GAS)
parking_lot.add_users('4', ParkingTipe.GAS)
parking_lot.add_users('Y', ParkingTipe.ELECTRIC)

parking_lot.add_users('Z', ParkingTipe.ELECTRIC)
parking_lot.add_users('5', ParkingTipe.GAS)
parking_lot.add_users('6', ParkingTipe.GAS)

parking_lot.add_users('Extra 7', ParkingTipe.GAS)
parking_lot.add_users('Extra S', ParkingTipe.ELECTRIC)
    
# Print Parking Lot
print_parking_lot(parking_lot)

# Someone pay, this guy is the first in
users_in = parking_lot.get_users()

us = users_in[0]
us.go_out()

print_parking_lot(parking_lot)

___________________________________________________________________________________________________________________________________
USERS -----------------------------------------------------------------------------------------------------------------------------
LVLS and AREAS --------------------------------------------------------------------------------------------------------------------
LVL: 0 
	 - n_areas: 3 
	 - Free: True 
	 - costs: 10

	AREA: 0 --> LVL :0 
		 - Type: ParkingTipe.GAS 
		 - Cost: 0 
		 - Free: True 
		 - User: None

	AREA: 1 --> LVL :0 
		 - Type: ParkingTipe.GAS 
		 - Cost: 0 
		 - Free: True 
		 - User: None

	AREA: 2 --> LVL :0 
		 - Type: ParkingTipe.ELECTRIC 
		 - Cost: 5 
		 - Free: True 
		 - User: None
LVL: 1 
	 - n_areas: 3 
	 - Free: True 
	 - costs: 10

	AREA: 0 --> LVL :1 
		 - Type: ParkingTipe.GAS 
		 - Cost: 0 
		 - Free: True 
		 - User: None

	AREA: 1 --> LVL :1 
		 - Type: ParkingTipe.GAS 
		 - Cost: 0 
		 - Free: True 
		 - User: None

	AREA

"\nus = User('xxx', ParkingTipe.GAS, parking_lot)\nus1 = User('xxx8888', ParkingTipe.GAS, parking_lot)\nus2 = User('xxx2222', ParkingTipe.GAS, parking_lot)\nus3 = User('xxx3333', ParkingTipe.GAS, parking_lot)\nus4 = User('666xxxx', ParkingTipe.ELECTRIC, parking_lot)\nus5 = User('666yyyy', ParkingTipe.ELECTRIC, parking_lot)\nus6 = User('666zzzz', ParkingTipe.ELECTRIC, parking_lot)\nus7 = User('xxx8888', ParkingTipe.GAS, parking_lot)\nus8 = User('xxx2222', ParkingTipe.GAS, parking_lot)\nus9 = User('xxx3333', ParkingTipe.GAS, parking_lot)"

---------------------------------
---------------------------------
---------------------------------
---------------------------------------- PARKING LOT OPTIMIZED ----------------------------------------
--------------------
---------------------------------
---------------------------------
---------------------------------

In [108]:
from enum import Enum
from datetime import datetime, timedelta

class ParkingType(Enum):
    GAS = 0
    ELECTRIC = 1

class Fee(Enum):
    DAY = 0
    NOON = 1
    NIGHT = 2

class ParkingArea:
    def __init__(self, id, level, type, cost):
        self.id = id
        self.level = level
        self.type = type
        self.cost = cost
        self.user = None
        self.free = True

    def is_free(self):
        return self.free

    def user_parks(self, user):
        if self.matching_types(user):
            self.user = user
            self.free = False
            user.set_time_in(datetime.now())
        else:
            print(f"You cannot park here; This is a {self.type.name} station, and you are a {user.type.name} user")
            return -1

        return 1

    def user_leaves(self):
        self.user.set_time_out(datetime.now())
        self.user.pay()
        self.user.area = None
        self.free = True
        self.user = None

    def matching_types(self, user):
        return user.type == self.type

    def get_cost(self):
        return self.cost

class User:
    def __init__(self, id_plate, type, parking_lot):
        self.id = id_plate
        self.type = type
        self.parking_lot = parking_lot
        self.time_in = None
        self.time_out = None
        self.level = None
        self.area = None

    def park(self):
        compatibility = self.get_area().user_parks(self)
        if compatibility == -1:
            return None

    def go_out(self):
        if self.parking_lot and self.area:
            self.area.user_leaves()
        else:
            print('You cannot go out because you are not in an AREA or PARKING LOT')

    def set_time_in(self, time):
        self.time_in = time

    def set_time_out(self, time):
        self.time_out = time

    def pay(self):
        parking_lot = self.parking_lot
        if parking_lot.get_cost() is None:
            print(f'Goodbye, Thanks for your visit.\nHope to see you again in {parking_lot.get_name()} ;)')
            self.level = None
            parking_lot.bye_user(self)
            self.parking_lot = None
            return
        else:
            lvl_cost = self.level.get_cost()
            area_cost = self.area.get_cost()
            parking_lot_hours = (self.time_out - self.time_in).total_seconds() / 3600
            cost = (lvl_cost + area_cost) * parking_lot_hours
            payed = 0
            while payed < cost:
                choice = float(input(f"You have to pay a total of {cost}; please insert coins $$: "))
                if isinstance(choice, (int, float)):
                    payed += choice

            print(f'Goodbye, Thanks for your visit.\nHope to see you again in {parking_lot.get_name()}\n\t don\'t forget your change: {payed - cost} ;)')

class Level:
    def __init__(self, level, num_parkings, num_gas_parkings, cost, gas_area_costs, electric_area_costs):
        self.level = level
        self.areas = [ParkingArea(i, level, ParkingType.GAS, gas_area_costs) if i < num_gas_parkings
                      else ParkingArea(i, level, ParkingType.ELECTRIC, electric_area_costs)
                      for i in range(num_parkings)]
        self.cost = cost

    def check_if_free(self):
        return any(area.is_free() for area in self.areas)

    def get_areas(self):
        return self.areas

    def get_cost(self):
        return self.cost

class ParkingLot:
    def __init__(self, name, num_levels, num_parkings, num_gas_parkings, level_costs, gas_area_costs, electric_area_costs, cost):
        self.name = name
        self.levels = [Level(i, num_parkings, num_gas_parkings, level_costs, gas_area_costs, electric_area_costs)
                       for i in range(num_levels)]
        self.users = []

    def get_name(self):
        return self.name

    def get_cost(self):
        return cost

    def add_level(self, level):
        self.levels.append(level)

    def bye_user(self, user):
        if user in self.users:
            self.users.remove(user)
        else:
            print(f'The user: {user.id}, Is not in the parking lot')

    def check_levels_free(self):
        return any(level.check_if_free() for level in self.levels)


def print_user(user):
    if user is not None:
        user_id = user.id if user.id is not None else "N/A"
        user_type = user.type if user.type is not None else "N/A"
        time_in = user.time_in if user.time_in is not None else "N/A"
        time_out = user.time_out if user.time_out is not None else "N/A"
        level = user.level if user.level is not None else "N/A"
        area_id = user.area.id if user.area is not None and user.area.id is not None else "N/A"

        print(f'USER: {user_id} \n\t - Type: {user_type} \n\t - time_in: {time_in} \n\t - time_out: {time_out} \n\t - level: {level} \n\t - area: {area_id}')
    else:
        print("Invalid user object")

# Rest of your code remains the same

def print_level(level):
    print(f'LVL: {level.level} \n\t - n_areas: {len(level.areas)} \n\t - Free: {level.check_if_free()} \n\t - costs: {level.cost}')


def print_area(area):
    print(f'\n\tAREA: {area.id} --> LVL: {area.level} \n\t\t - Type: {area.type} \n\t\t - Cost: {area.cost} \n\t\t - Free: {area.free} \n\t\t - User: {area.user}')


def print_parking_lot(parking_lot):
    print('___________________________________________________________________________________________________________________________________')
    print('USERS -----------------------------------------------------------------------------------------------------------------------------')

    for u in parking_lot.users:
        print_user(u)

    print('LVLS and AREAS --------------------------------------------------------------------------------------------------------------------')
    for l in parking_lot.levels:
        print_level(l)
        for a in l.areas:
            print_area(a)

    print('___________________________________________________________________________________________________________________________________')


n_levels = 3
n_parkings = 3
n_gas_parkings = 2

level_cost = 5
gas_area_costs = 0
electric_area_costs = 5
cost = 10

parking_lot = ParkingLot('AYAWASKA WEY', n_levels, n_parkings, n_gas_parkings, level_cost, gas_area_costs, electric_area_costs, cost)
print_parking_lot(parking_lot)
print('\n\n')

# Make Parking Lot Full

parking_lot.users.append(User('1', ParkingType.GAS, parking_lot))
parking_lot.users.append(User('2', ParkingType.GAS, parking_lot))
parking_lot.users.append(User('X', ParkingType.ELECTRIC, parking_lot))

parking_lot.users.append(User('3', ParkingType.GAS, parking_lot))
parking_lot.users.append(User('4', ParkingType.GAS, parking_lot))
parking_lot.users.append(User('Y', ParkingType.ELECTRIC, parking_lot))

parking_lot.users.append(User('Z', ParkingType.ELECTRIC, parking_lot))
parking_lot.users.append(User('5', ParkingType.GAS, parking_lot))
parking_lot.users.append(User('6', ParkingType.GAS, parking_lot))

parking_lot.users.append(User('Extra 7', ParkingType.GAS, parking_lot))
parking_lot.users.append(User('Extra S', ParkingType.ELECTRIC, parking_lot))

# Print Parking Lot
print_parking_lot(parking_lot)

# Someone pays, this guy is the first in
users_in = parking_lot.users

us = users_in[0]
us.set_time_in(datetime.now())
us.go_out()

print_parking_lot(parking_lot)

___________________________________________________________________________________________________________________________________
USERS -----------------------------------------------------------------------------------------------------------------------------
LVLS and AREAS --------------------------------------------------------------------------------------------------------------------
LVL: 0 
	 - n_areas: 3 
	 - Free: True 
	 - costs: 5

	AREA: 0 --> LVL: 0 
		 - Type: ParkingType.GAS 
		 - Cost: 0 
		 - Free: True 
		 - User: None

	AREA: 1 --> LVL: 0 
		 - Type: ParkingType.GAS 
		 - Cost: 0 
		 - Free: True 
		 - User: None

	AREA: 2 --> LVL: 0 
		 - Type: ParkingType.ELECTRIC 
		 - Cost: 5 
		 - Free: True 
		 - User: None
LVL: 1 
	 - n_areas: 3 
	 - Free: True 
	 - costs: 5

	AREA: 0 --> LVL: 1 
		 - Type: ParkingType.GAS 
		 - Cost: 0 
		 - Free: True 
		 - User: None

	AREA: 1 --> LVL: 1 
		 - Type: ParkingType.GAS 
		 - Cost: 0 
		 - Free: True 
		 - User: None

	AREA: 

---------------------------------------------------------
--------------------------------------------------------
---------------------------------------------------------
----------------- More Optimized -------------------------
-----------------
------------------------------------------------------
-----------------------------------------------------
------------------------------------------------------


In [134]:
from enum import Enum

class VehicleSize(Enum):
    Motorcycle = 1
    Compact = 2
    Large = 3

class Vehicle:
    def __init__(self, license_plate):
        self.parking_spots = []
        self.license_plate = license_plate
        self.spots_needed = 0
        self.size = None

    def get_spots_needed(self):
        return self.spots_needed

    def get_size(self):
        return self.size

    def park_in_spot(self, spot):
        self.parking_spots.append(spot)

    def clear_spots(self):
        for spot in self.parking_spots:
            spot.remove_vehicle()
        self.parking_spots = []

    def can_fit_in_spot(self, spot):
        return self.get_size() >= spot.spot_size  # Compare vehicle size with spot size

class Bus(Vehicle):
    def __init__(self, license_plate):
        super().__init__(license_plate)
        self.spots_needed = 5
        self.size = VehicleSize.Large

    def can_fit_in_spot(self, spot):
        return spot.spot_size == VehicleSize.Large

class Car(Vehicle):
    def __init__(self, license_plate):
        super().__init__(license_plate)
        self.spots_needed = 1
        self.size = VehicleSize.Compact

    def can_fit_in_spot(self, spot): 
        return spot.spot_size in (VehicleSize.Compact, VehicleSize.Large)

class Motorcycle(Vehicle):
    def __init__(self, license_plate):
        super().__init__(license_plate)
        self.spots_needed = 1
        self.size = VehicleSize.Motorcycle

    def can_fit_in_spot(self, spot):
        return spot.spot_size == VehicleSize.Motorcycle

class ParkingSpot:
    def __init__(self, level, row, number, spot_size):
        self.vehicle = None
        self.spot_size = spot_size
        self.row = row
        self.spot_number = number
        self.level = level

    def is_available(self):
        return self.vehicle is None

    def can_fit_vehicle(self, vehicle):
        return self.is_available() and vehicle.can_fit_in_spot(self)

    def park(self, vehicle):
        if self.can_fit_vehicle(vehicle):
            self.vehicle = vehicle
            vehicle.park_in_spot(self)

    def remove_vehicle(self):
        if self.vehicle:
            self.vehicle = None

class Level:
    def __init__(self, floor, number_spots):
        self.floor = floor
        self.spots = []
        self.available_spots = number_spots
        self.SPOTS_PER_ROW = 10
        self.initialize_spots(number_spots)

    def available_spots(self):
        return self.available_spots

    def initialize_spots(self, number_spots):
        spot_size = VehicleSize.Motorcycle
        for i in range(number_spots):
            if i == number_spots // 2:
                spot_size = VehicleSize.Compact
            if i == number_spots - 5:
                spot_size = VehicleSize.Large
            row = i // self.SPOTS_PER_ROW
            spot = ParkingSpot(self, row, i % self.SPOTS_PER_ROW, spot_size)
            self.spots.append(spot)

    def park_vehicle(self, vehicle):
        if self.available_spots == 0:
            return False  # No spots available

        # Try to find a spot that can fit the vehicle
        for spot in self.spots:
            if spot.can_fit_vehicle(vehicle):
                spot.park(vehicle)
                self.available_spots -= vehicle.get_spots_needed()
                return True

        return False  # No suitable spot found for the vehicle

class ParkingLot:
    def __init__(self, num_levels):
        self.levels = [Level(i, 30) for i in range(num_levels)]

    def park_vehicle(self, vehicle):
        for level in self.levels:
            if level.park_vehicle(vehicle):
                return True
        return False

In [138]:
def test_parking_lot():
    parking_lot = ParkingLot(num_levels=2)

    # Create vehicles
    bus = Bus("BUS123")
    car1 = Car("CAR456")
    motorcycle = Motorcycle("MOTO789")
    car2 = Car("CAR101")
    
    # Park the vehicles
    assert parking_lot.park_vehicle(bus) == True
    assert parking_lot.park_vehicle(car1) == True
    assert parking_lot.park_vehicle(motorcycle) == True
    assert parking_lot.park_vehicle(car2) == True
    
    # Try parking an oversized vehicle
    #oversized_vehicle = Bus("OVERSIZED")
    #assert parking_lot.park_vehicle(oversized_vehicle) == False

    # Clear spots and check availability
    car1.clear_spots()
    #assert parking_lot.park_vehicle(oversized_vehicle) == True

    # Unpark vehicles
    car2.clear_spots()
    bus.clear_spots()
    motorcycle.clear_spots()
    #oversized_vehicle.clear_spots()
    
    print("All test cases passed!")

if __name__ == "__main__":
    test_parking_lot()

All test cases passed!


#### Exercise 7.5
**Online Book Reader:** Design the data structures for an online book reader system.

**Hints:**
- #344: Think about all the different functionality a system to read books online would have to support. You don't have to do everything, but you should think about making your assumptions explicit. 

In [139]:
from enum import Enum

In [343]:
class Reader:
    def __init__(self, name, password):
        self.name = name
        self.password = password
        self.favorite_books = []
        self.reading_book = None
        self.logged = False
        self.finish = []
        
    def print_user(self):
        print(f"User Name: {self.get_name()} - password: {self.get_password()} - get_favorite_books: {self.get_favorite_books()} - get_reading_book: {self.get_reading_book()} --- logged: {self.get_logged()}")

    def get_name(self):
        return self.name
    
    def get_password(self):
        return self.password
        
    def get_favorite_books(self):
        return self.favorite_books
        
    def add_fav_book(self, book):
        self.favorite_books.append(book)

    def get_reading_book(self):
        return self.reading_book

    def start_reading_book(self, book):
        self.reading_book = book
        
    def get_logged(self):
        return self.logged
        
    def log_in(self):
        self.logged = True

    def log_out(self):
        self.logged = False

    

class BookType(Enum):
    FANTASY = 0
    PSYCHOLOGY = 1
    PHILOSOPHY = 2
    ANIME = 3
    CRIMES = 4

class Page:
    def __init__(self, content, n_page):
        self.content = content
        self.n_page = n_page

    def get_content(self):
        return self.content

    def get_n_page(self):
        return self.n_page

class Book:
    def __init__(self, name, author, n_pages, book_type, library = None):
        self.name = name
        self.author = author
        self.n_pages = n_pages
        self.book_type = book_type
        #self.reading = False
        #self.finish = False
        self.current_page = 0
        self.pages = [Page(f'xxx{n_p}', n_p) for n_p in range(n_pages)]
        self.library = library

    def pr_book(self):
        print(f" Book Name: {self.get_name()} - Author: {self.get_author()} - Npages: {self.get_n_pages()} - Type: {self.get_book_type()} --- Library: {self.get_library().get_name()}")
        self.show_page()

    def get_library(self):
        return self.library
        
    def get_name(self):
        return self.name

    def get_author(self):
        return self.author

    def get_pages(self):
        return self.pages

    def get_n_pages(self):
        return self.n_pages

    def get_book_type(self):
        return self.book_type

    def get_reading(self):
        return self.reading
    
    def currently_reading(self):
        self.reading = True

    def get_finish(self):
        return self.finish

    def finished(self):
        self.finish = True
        self.reading = False
        self.current_page = self.n_pages

    def show_page(self):
        print(f'\t\t\t\t - Page: {self.pages[self.current_page].get_n_page()} --- \n \t {self.pages[self.current_page].get_content()}')
        
    def read_page(self, user):
        if user.get_reading_book() == self:
            self.currently_reading()

        if self.current_page < self.get_n_pages():
            print(f'\t\t\t\t - Page: {self.pages[self.current_page].get_n_page()} --- \n \t {self.pages[self.current_page].get_content()}')
            self.current_page += 1
        else:
            print('Book Ended')
            if user.reading_book != None:
                user.reading_book = None
                user.finish.append(self)
            

class Library:
    def __init__(self, name):
        self.name = name
        self.books = []
        self.n_books = 0

    def add_book(self, name, author, n_pages, book_type):
        self.books.append(Book(name, author, n_pages, book_type, self))
        self.n_books += 1

    def get_name(self):
        return self.name

    def get_n_books(self):
        return self.n_books
    # def remove_book(self, book)

    def get_books(self):
        return self.books

    def search_book(self, book):
        if book in self.books:
            return True
        return False

    def search_book_name(self, book_name):
        if book_name in [books.get_name() for books in self.books]:
            return True
        return False

    def search_books_type(self, book_type):
        bt = [books for books in self.books if books.get_book_type() == book_type]
        return bt # Esto podría devolver una librería con el número de líbros

    def print_library(self):
        print('\t - ', self.get_name(), f' - n_books: {self.get_n_books()}:')
        for i, b in enumerate(self.get_books()):
            print(f"\t\t {i}.- Book Name: {b.get_name()} - Author: {b.get_author()} - Npages: {b.get_n_pages()} - Type: {b.get_book_type()} --- Library: {b.get_library().get_name()}")

class BookReader:
    def __init__(self, name):
        self.name = name
        self.libraries = []
        self.users_dic = {}
        self.users = []
        self.user_loged = None

    def get_users(self):
        return self.users
        
    def get_dic(self):
        return self.users_dic
        
    def get_name(self):
        return self.name

    def add_library(self, library):
        self.libraries.append(library)

    def get_libraries(self):
        return self.libraries

    def search_book(self, book):
        for l in self.libraries:
            if l.search_book(book) == True:
                return l, True
            return False

    def search_book_name(self, book_name):
        for l in self.libraries:
            if l.search_book_name(book_name) == True:
                return l, True
            return False

    def search_books_type(self, book_type):
        lbt = []
        for l in self.libraries:
            bt = l.search_books_type(book_type)
            if bt != []:
                lbt.append(bt)
        if lbt == []:
            print('No Books of that category found')
            return None
        
        return lbt
        
    def register_user(self, name, password):
        if name not in self.users_dic.keys():
            us = Reader(name, password)
            self.users_dic[name] = password
            self.users.append(us)
        else:
            print('Name already Registered')
    
    def login_user(self, user):
        if user.name in self.users_dic.keys():
            if user.password == self.users_dic[user.name]:
                user.log_in()
                if self.user_loged == None:
                    self.user_loged = user
                else:
                    print('already a user logged')
            else:
                print('Wrong Password')
        else:
            print('Wrong Username')

    def logout(self):
        self.user_loged.log_out()
        self.user_loged = None

    def print_bookreader(self):
        print(self.get_name(), ':')
        for i, l in enumerate(self.libraries):
            print(f'\t - Library: {i} -- {l.get_name()}')

    def print_bookreader_users(self):
        print(self.get_name(), ':')
        for i, l in enumerate(self.get_dic().keys()):
            print(f'\t - User {i}: -- {l}: pass: {self.get_dic()[l]}')

In [346]:
library = Library('Animation')
library.add_book('jujutsu - 0', 'Sensei', 10, BookType.ANIME)
library.add_book('jujutsu - 1', 'Sensei', 10, BookType.ANIME)
library.add_book('jujutsu - 2', 'Sensei', 10, BookType.ANIME)
library.add_book('jujutsu - 3', 'Sensei', 10, BookType.ANIME)
library.add_book('jujutsu - 4', 'Sensei', 10, BookType.ANIME)
library.add_book('jujutsu - 5', 'Sensei', 10, BookType.ANIME)
library.add_book('jujutsu - 6', 'Sensei', 10, BookType.ANIME)
library.add_book('jujutsu - 7', 'Sensei', 10, BookType.ANIME)
library.add_book('jujutsu - 8', 'Sensei', 10, BookType.ANIME)

library.add_book('1.- El Nombre del Viento', 'Rodfuks', 8, BookType.FANTASY)
library.add_book('2.- El Temor de un Hombre Sabio', 'Rodfuks', 9, BookType.FANTASY)
library.add_book('3.- Las Puertas de Piedra', 'Rodfuks', 10, BookType.FANTASY)

library2 = Library('Diario De Bitacora')
library2.add_book('100 Questions of Life', 'Platon', 11, BookType.PSYCHOLOGY)
library2.add_book('Im Becoming a God', 'Seneca', 11, BookType.PSYCHOLOGY)
library2.add_book('Fight for your Life', 'Epicteto', 11, BookType.PHILOSOPHY)
library2.add_book('Dont know, Dont Care ', 'Myself', 11, BookType.PHILOSOPHY)

mappa = BookReader('La Casa Del Libro')

mappa.add_library(library)
mappa.add_library(library2)

mappa.print_bookreader()
print()

libraries = mappa.get_libraries()
for l in libraries:
    l.print_library()
    print()

# Register users:
mappa.register_user('Gojo', '8888')
mappa.register_user('Gojo', '9999')
mappa.register_user('Saitama', '9999')
mappa.print_bookreader_users()
print()

users = mappa.get_users()
for u in users:
    u.print_user()
print()

## Login 
gojo = users[0]

mappa.login_user(gojo)
mappa.login_user(users[1])
print()

## Make Functions with the logged User:
gojo.print_user()
print()

## Search Books of ANIME:
#Books
juju = library.get_books()[0]
juju8 = library.get_books()[8]

# Searchs
print(mappa.search_book(juju))
print(mappa.search_book_name('jujutsu - 8'))
print(mappa.search_books_type(BookType.ANIME))
searched_list = mappa.search_books_type(BookType.ANIME)
flattened_list = [item for sublist in searched_list for item in sublist]
for s in flattened_list:
    s.pr_book()

# Add To Favorite Books
gojo.add_fav_book(juju)
gojo.add_fav_book(juju8)

# Show Favorite Books
print(gojo.get_favorite_books())
print()

# Read some pages of a book
gojo.start_reading_book(juju)
print(gojo.get_reading_book().get_name())
print()

gojo.get_reading_book().read_page(gojo)
gojo.get_reading_book().read_page(gojo)
gojo.get_reading_book().read_page(gojo)
gojo.get_reading_book().read_page(gojo)
gojo.get_reading_book().read_page(gojo)
gojo.get_reading_book().read_page(gojo)
gojo.get_reading_book().read_page(gojo)
print()

# Read till the end
gojo.get_reading_book().read_page(gojo)
gojo.get_reading_book().read_page(gojo)
gojo.get_reading_book().read_page(gojo)
gojo.get_reading_book().read_page(gojo)
# gojo.get_reading_book().read_page(gojo)

## Logout
mappa.logout()


La Casa Del Libro :
	 - Library: 0 -- Animation
	 - Library: 1 -- Diario De Bitacora

	 -  Animation  - n_books: 12:
		 0.- Book Name: jujutsu - 0 - Author: Sensei - Npages: 10 - Type: BookType.ANIME --- Library: Animation
		 1.- Book Name: jujutsu - 1 - Author: Sensei - Npages: 10 - Type: BookType.ANIME --- Library: Animation
		 2.- Book Name: jujutsu - 2 - Author: Sensei - Npages: 10 - Type: BookType.ANIME --- Library: Animation
		 3.- Book Name: jujutsu - 3 - Author: Sensei - Npages: 10 - Type: BookType.ANIME --- Library: Animation
		 4.- Book Name: jujutsu - 4 - Author: Sensei - Npages: 10 - Type: BookType.ANIME --- Library: Animation
		 5.- Book Name: jujutsu - 5 - Author: Sensei - Npages: 10 - Type: BookType.ANIME --- Library: Animation
		 6.- Book Name: jujutsu - 6 - Author: Sensei - Npages: 10 - Type: BookType.ANIME --- Library: Animation
		 7.- Book Name: jujutsu - 7 - Author: Sensei - Npages: 10 - Type: BookType.ANIME --- Library: Animation
		 8.- Book Name: jujutsu - 8 - Aut

----------------------------------------------
------------------------------
---------------------------------
-------------------OPTIMIZED ------------------------
------
----------------------------
--------------------------
------------------------

In [347]:
from enum import Enum

# Define BookType enum
class BookType(Enum):
    FANTASY = 0
    PSYCHOLOGY = 1
    PHILOSOPHY = 2
    ANIME = 3

class Book:
    def __init__(self, title, author, total_pages, book_type):
        self.title = title
        self.author = author
        self.total_pages = total_pages
        self.book_type = book_type

class Library:
    def __init__(self, name):
        self.name = name
        self.books = []

    def add_book(self, book):
        self.books.append(book)

    def search_books_by_type(self, book_type):
        return [book for book in self.books if book.book_type == book_type]

class User:
    def __init__(self, name, password):
        self.name = name
        self.password = password
        self.favorite_books = []
        self.current_reading = {}

    def add_to_favorites(self, book):
        self.favorite_books.append(book)

    def start_reading(self, book, page):
        if book.title in self.current_reading:
            print("You are already reading this book.")
        else:
            self.current_reading[book.title] = page

    def finish_reading(self, book):
        if book.title in self.current_reading:
            del self.current_reading[book.title]
        else:
            print("You have not started reading this book.")

class BookReaderSystem:
    def __init__(self):
        self.libraries = []
        self.users = []
        self.logged_in_user = None

    def register_user(self, name, password):
        if any(user.name == name for user in self.users):
            print("Username already exists. Please choose a different username.")
        else:
            user = User(name, password)
            self.users.append(user)
            print(f"User '{name}' registered successfully.")

    def login(self, user):
        if self.logged_in_user:
            print("Another user is already logged in.")
        else:
            if user in self.users:
                self.logged_in_user = user
                print(f"User '{user.name}' logged in.")
            else:
                print("Invalid username or password.")

    def logout(self):
        if self.logged_in_user:
            print(f"User '{self.logged_in_user.name}' logged out.")
            self.logged_in_user = None
        else:
            print("No user is currently logged in.")

    def search_books(self, book_type):
        matching_books = []
        for library in self.libraries:
            matching_books.extend(library.search_books_by_type(book_type))
        return matching_books

# Test the Online Book Reader system
if __name__ == "__main__":
    # Create libraries
    library1 = Library("Fantasy Library")
    library2 = Library("Psychology Library")

    # Add books to libraries
    book1 = Book("The Hobbit", "J.R.R. Tolkien", 300, BookType.FANTASY)
    book2 = Book("The Alchemist", "Paulo Coelho", 200, BookType.PHILOSOPHY)
    library1.add_book(book1)
    library2.add_book(book2)

    # Create the Book Reader System
    reader_system = BookReaderSystem()

    # Register users
    reader_system.register_user("Alice", "password123")
    reader_system.register_user("Bob", "password456")

    # Login as Alice
    reader_system.login(reader_system.users[0])

    # Alice adds a book to her favorites
    alice = reader_system.logged_in_user
    alice.add_to_favorites(book1)

    # Alice starts reading a book
    alice.start_reading(book1, 1)

    # Search for books
    fantasy_books = reader_system.search_books(BookType.FANTASY)
    print(f"Fantasy Books: {len(fantasy_books)} found")

    # Logout Alice
    reader_system.logout()

    # Login as Bob
    reader_system.login(reader_system.users[1])

    # Bob tries to read the same book as Alice
    bob = reader_system.logged_in_user
    bob.start_reading(book1, 1)  # Should print "You are already reading this book."

    # Bob adds a book to his favorites
    bob.add_to_favorites(book2)

    # Bob starts reading a book
    bob.start_reading(book2, 1)

    # Logout Bob
    reader_system.logout()

    # Login as Alice again
    reader_system.login(reader_system.users[0])

    # Alice finishes reading the book
    alice.finish_reading(book1)

    # Logout Alice
    reader_system.logout()

User 'Alice' registered successfully.
User 'Bob' registered successfully.
User 'Alice' logged in.
Fantasy Books: 0 found
User 'Alice' logged out.
User 'Bob' logged in.
User 'Bob' logged out.
User 'Alice' logged in.
User 'Alice' logged out.


#### Exercise 7.6
**Jigsaw:** Implement an NxN jigsaw puzzle. Design the data structures and explain an algorithm to solve the puzzle. You can assume that you have a fitsWith method which, when passed two puzzle edges, returns true if the two edges belong together.

**Hints:**
- #192: A common trick when solving a jigsaw puzzle is to separate edge and non-edge pieces. How will you represent this in an object-oriented manner?
- #238: Think about how you might record the position of a piece when you find it. Should it be stored by row and location? 
- #283: Which will be the easiest pieces to match first? Can you start with those? Which will be the next easiest, once you've nailed those down?

In [548]:
from enum import Enum
import random
from collections import deque

In [552]:
from enum import Enum
from collections import deque

class Orientation(Enum):
    LEFT = 0
    TOP = 1
    RIGHT = 2
    BOTTOM = 3

    def get_opposite(self):
        if self == Orientation.LEFT:
            return Orientation.RIGHT
        elif self == Orientation.RIGHT:
            return Orientation.LEFT
        elif self == Orientation.TOP:
            return Orientation.BOTTOM
        elif self == Orientation.BOTTOM:
            return Orientation.TOP
        else:
            return None

class Shape(Enum):
    INNER = 0
    OUTER = 1
    FLAT = 2

    def get_opposite(self):
        if self == Shape.INNER:
            return Shape.OUTER
        elif self == Shape.OUTER:
            return Shape.INNER
        else:
            return None

class Edge:
    def __init__(self, shape):
        self.shape = shape
        self.parent_piece = None

    def fits_with(self, edge):
        return self.shape == edge.shape

class Piece:
    def __init__(self, edge_list):
        self.edges = {
            Orientation.LEFT: edge_list[0],
            Orientation.TOP: edge_list[1],
            Orientation.RIGHT: edge_list[2],
            Orientation.BOTTOM: edge_list[3]
        }

    def rotate_edges_by(self, number_rotations):
        number_rotations = number_rotations % 4
        for _ in range(number_rotations):
            temp = self.edges[Orientation.LEFT]
            self.edges[Orientation.LEFT] = self.edges[Orientation.TOP]
            self.edges[Orientation.TOP] = self.edges[Orientation.RIGHT]
            self.edges[Orientation.RIGHT] = self.edges[Orientation.BOTTOM]
            self.edges[Orientation.BOTTOM] = temp

class Puzzle:
    def __init__(self, size, pieces):
        self.pieces = pieces
        self.solution = [[None] * size for _ in range(size)]
        self.size = size

    def set_edge_in_solution(self, edge, row, column, orientation):
        piece = edge.parent_piece
        piece.rotate_edges_by(orientation.value - 1)
        self.pieces.remove(piece)
        self.solution[row][column] = piece

    def fit_next_edge(self, pieces_to_search, row, col):
        if not pieces_to_search:
            return False  # No pieces left to search

        if row == 0 and col == 0:
            p = pieces_to_search.popleft()
            self.orient_top_left_corner(p)
            self.solution[0][0] = p
        else:
            piece_to_match = self.solution[row - 1][0] if col == 0 else self.solution[row][col - 1]
            orientation_to_match = Orientation.BOTTOM if col == 0 else Orientation.RIGHT
            edge_to_match = piece_to_match.edges[orientation_to_match]
            edge = self.get_matching_edge(edge_to_match, pieces_to_search)
            if edge is None:
                return False  # Can't solve
            orientation = orientation_to_match.get_opposite()
            self.set_edge_in_solution(edge, row, col, orientation)
        return True

    def solve(self):
        corner_pieces = []
        border_pieces = []
        inside_pieces = []
        self.group_pieces(corner_pieces, border_pieces, inside_pieces)
        self.solution = [[None] * self.size for _ in range(self.size)]
        for row in range(self.size):
            for col in range(self.size):
                pieces_to_search = self.get_piece_list_to_search(corner_pieces, border_pieces, inside_pieces, row, col)
                if not self.fit_next_edge(pieces_to_search, row, col):
                    return False
        return True

    def orient_top_left_corner(self, piece):
        while piece.edges[Orientation.TOP].shape != Shape.FLAT or piece.edges[Orientation.LEFT].shape != Shape.FLAT:
            piece.rotate_edges_by(1)

    def group_pieces(self, corner_pieces, border_pieces, inside_pieces):
        # Implement this method to group pieces as per your puzzle setup.
        # You should populate corner_pieces, border_pieces, and inside_pieces lists.
        pass

    def get_matching_edge(self, edge_to_match, pieces_to_search):
        for piece in pieces_to_search:
            for orientation, edge in piece.edges.items():
                if edge.fits_with(edge_to_match):
                    return edge
        return None

    def get_piece_list_to_search(self, corner_pieces, border_pieces, inside_pieces, row, col):
        # Implement this method to return the appropriate list of pieces to search based on the row and column.
        pass

# Test the Puzzle class
if __name__ == "__main__":
    # Create a list of pieces (you should implement this based on your puzzle setup)
    pieces_list = []

    # Create a puzzle with a size and the list of pieces
    puzzle = Puzzle(3, deque(pieces_list))

    # Attempt to solve the puzzle
    if puzzle.solve():
        print("Puzzle solved!")
        for row in puzzle.solution:
            print([piece is not None for piece in row])
    else:
        print("Cannot solve the puzzle.")

Cannot solve the puzzle.


------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------

In [544]:
class Ficha:
    def __init__(self, x, y, n):
        self.x = x
        self.y = y
        self.n = n

    def get_x(self):
        return self.x

    def set_x(self, x):
        self.x = x
        
    def get_y(self):
        return self.y

    def set_y(self, y):
        self.y = y

    def is_corner(self):
        if self.x == 0 and self.y == 0:
            return UPL
        elif self.x == 0 and self.y == self.n-1:
            return UPR
        elif self.x == self.n-1 and self.y == 0:
            return DWL
        elif self.x == self.n-1 and self.y == self.n-1:
            return DWR
        return False
        
    def is_border(self):
        if self.x == 0 and self.y == 0:
            return UP
        elif self.x == 0 and self.y == self.n-1:
            return L
        elif self.x == self.n-1 and self.y == 0:
            return DW
        elif self.x == self.n-1 and self.y == self.n-1:
            return R
        return False

class Jigsaw:
    def __init__(self, name, n):
        self.name = name
        self.fichas = []
        for r in range(n):
            l = []
            for i in range(n):
                l.append(Ficha(i,r,n))
            self.fichas.append(l)
        self.random_fichas = [f for r in self.fichas for f in r]
        random.shuffle(self.random_fichas)
        self.tam = n

    def get_name(self):
        return self.name

    def get_tam(self):
        return self.tam

    # Solution
    def get_fichas(self):
        return self.fichas
        
    # Start game with random piezas
    def get_random(self):
        return self.random_fichas

class Player:
    def __init__(self, name, jigsaw):
        self.name = name
        self.jigsaw = jigsaw
        self.random_piezas = jigsaw.get_random()
        self.solve_jigsaw = []
        for _ in range(jigsaw.get_tam()):
            l = []
            for _ in range(jigsaw.get_tam()):
                l.append(None)
            self.solve_jigsaw.append(l)
        self.its = 0

    def get_its(self):
        return self.its

    def set_its(self, its):
        self.its = its
        
    def get_name(self):
        return name

    def get_jigsaw(self):
        return self.jigsaw

    def get_org_jigsaw(self):
        return self.jigsaw

    def get_random_piezas(self):
        return self.random_piezas

    def get_solve_jigsaw(self):
        return self.solve_jigsaw

    # Fits With
    def fitsWith(self, pieza, pieza2):
        return abs(pieza.get_x() - pieza2.get_x()) == 1 and pieza.get_y() == pieza2.get_y() or abs(pieza.get_y() - pieza2.get_y()) == 1 and pieza.get_x() == pieza2.get_x()
        
    '''def set_solve_esquinas(self):
        corners = []
        for rp in self.get_random_piezas():
            if rp.is_esquina() != False:
                if rp.is_esquina() == Esquina.UPL:
                    corner.append(rp)
                    self.get_solve_jigsaw()[self.get_its()][self.get_its()] = rp
                elif rp.is_esquina() == Esquina.UPR:
                    self.get_solve_jigsaw()[self.get_its()][self.get_jigsaw().get_tam() - self.get_its() - 1] = rp
                    corner.append(rp)
                elif rp.is_esquina() == Esquina.DWL:
                    self.get_solve_jigsaw()[self.get_jigsaw().get_tam() - self.get_its() -1][self.get_its()] = rp
                    corner.append(rp)
                elif rp.is_esquina() == Esquina.DWR:
                    self.get_solve_jigsaw()[self.get_jigsaw().get_tam() - self.get_its() - 1][self.get_jigsaw().get_tam() - self.get_its() - 1] = rp
                    corner.append(rp)

        # Clean Corners from possible selectable pieces
        for r in corner:
            self.get_random_piezas().pop(r)'''

    def set_solve_esquinas(self):
        corner = []
        print('CORNERS: ')
        for rp in self.get_random_piezas():
            # Up L
            if rp.get_x() == self.get_its() and rp.get_y() == self.get_its():
                self.get_solve_jigsaw()[self.get_its()][self.get_its()] = rp
                print('\t - upl rp', rp, ' x ', rp.get_x(), ' y ', rp.get_y())
                corner.append(rp)
            # Up R
            if rp.get_x() == (self.get_jigsaw().get_tam() - self.get_its() - 1) and rp.get_y() == self.get_its():
                self.get_solve_jigsaw()[self.get_its()][self.get_jigsaw().get_tam() - self.get_its() - 1] = rp
                print('\t - upr rp', rp, ' x ', rp.get_x(), ' y ', rp.get_y())
                corner.append(rp)
            # Dw L
            if rp.get_x() == self.get_its() and rp.get_y() == (self.get_jigsaw().get_tam() - self.get_its() - 1):
                self.get_solve_jigsaw()[self.get_jigsaw().get_tam() - self.get_its() -1][self.get_its()] = rp
                print('\t - downl rp', rp, ' x ', rp.get_x(), ' y ', rp.get_y())
                corner.append(rp)
            # Dw R
            elif rp.get_x() == (self.get_jigsaw().get_tam() - self.get_its() - 1) and rp.get_y() == (self.get_jigsaw().get_tam() - self.get_its() - 1):
                self.get_solve_jigsaw()[self.get_jigsaw().get_tam() - self.get_its() - 1][self.get_jigsaw().get_tam() - self.get_its() - 1] = rp
                print('\t - downr rp', rp, ' x ', rp.get_x(), ' y ', rp.get_y())
                corner.append(rp)

        # Clean Corners from possible selectable pieces
        for r in corner:
            self.get_random_piezas().pop(self.get_random_piezas().index(r))

    def get_borders(self):
        up, l, dw, r = [], [], [], []
        for p in self.get_random_piezas():
            # Up
            if p.get_y() == self.get_its():
                up.append(p)
            elif p.get_x() == self.get_its():
                l.append(p)
            elif p.get_y() == self.get_jigsaw().get_tam() - self.get_its() - 1:
                dw.append(p)
            elif p.get_x() == self.get_jigsaw().get_tam() - self.get_its() - 1:
                r.append(p)

        if len(up) != self.get_jigsaw().get_tam() - self.get_its() - 2:
            print('something is wrong with borders len: ', len(up))

        return up, l, dw, r
                
    def set_solve_bordes(self): 
        up, l, dw, e = self.get_borders()
        print(len(up), len(l), len(dw), len(e))
        print(up)
        print(l)
        print(dw)
        print(e)
        
        # Make UP Borders
        i = 0
        counter = 0
        print('BORDERS: ')
        while i != self.get_jigsaw().get_tam() - self.get_its() - 1:
            if self.fitsWith(up[counter], self.get_solve_jigsaw()[0][i]):
                print('UP i:', i)
                print('\t -  counter UP = ', up[counter], ' c = ', counter)
                print('\t - solve_jogsaw i:', i, ' - ', self.get_solve_jigsaw()[0][i], ' x ', self.get_solve_jigsaw()[0][i].get_x(), ' y ', self.get_solve_jigsaw()[0][i].get_y())

                self.set_pieza(up[counter], 0, i+1)
                i += 1
                counter = 0
            counter += 1
        # Delete UP piezas:
        for u in up:
            self.get_random_piezas().pop(self.get_random_piezas().index(u))

        # Make LEFT Borders
        i = 0
        counter = 0
        while i != self.get_jigsaw().get_tam() - self.get_its() - 1:
            if self.fitsWith(l[counter], self.get_solve_jigsaw()[i][0]):
                print('LEFT i:', i)
                print('\t -  counter LEFT = ', l[counter], ' c = ', counter)
                print('\t - solve_jogsaw i:', i, ' - ', self.get_solve_jigsaw()[i][0], ' x ', self.get_solve_jigsaw()[i][0].get_x(), ' y ', self.get_solve_jigsaw()[i][0].get_y())
            
                self.set_pieza(l[counter], i+1, 0)
                i += 1
                counter = 0
            counter += 1
        # Delete LEFT piezas:
        for e in l:
            self.get_random_piezas().pop(self.get_random_piezas().index(e))

        # Make DOWN Borders
        i = 0
        counter = 0
        while i != self.get_jigsaw().get_tam() - self.get_its() - 1:
            print('DW', dw)
            print('C', counter)
            print('DWC', dw[counter])
            print()
            print('len', len(dw))
            print()
            if self.fitsWith(dw[counter], self.get_solve_jigsaw()[self.get_jigsaw().get_tam() - self.get_its() - 1][i]):
                print('DOWN i:', i)
                print('\t -  counter LEFT = ', dw[counter], ' c = ', counter)
                print('\t - solve_jogsaw i:', i, ' - ', self.get_solve_jigsaw()[self.get_jigsaw().get_tam() - self.get_its() - 1][i], ' x ', self.get_solve_jigsaw()[self.get_jigsaw().get_tam() - self.get_its() - 1][i].get_x(), ' y ', self.get_solve_jigsaw()[self.get_jigsaw().get_tam() - self.get_its() - 1][i].get_y())
                self.set_pieza(dw[counter], self.get_jigsaw().get_tam() - self.get_its() - 1, i+1)
                i += 1
                counter = 0
            counter += 1
        # Delete DOWN piezas:
        for d in dw:
            self.get_random_piezas().pop(self.get_random_piezas().index(d))

        # Make RIGHT Borders
        i = 0
        counter = 0
        while i != self.get_jigsaw().get_tam() - self.get_its() - 1:
            if self.fitsWith(r[counter], self.get_solve_jigsaw()[i][self.get_jigsaw().get_tam() - self.get_its() - 1]):
                print('RIGHT i:', i)
                print('\t -  counter LEFT = ', r[counter], ' c = ', counter)
                print('\t - solve_jogsaw i:', i, ' - ', self.get_solve_jigsaw()[i][self.get_jigsaw().get_tam() - self.get_its() - 1], ' x ', self.get_solve_jigsaw()[i][self.get_jigsaw().get_tam() - self.get_its() - 1].get_x(), ' y ', self.get_solve_jigsaw()[i][self.get_jigsaw().get_tam() - self.get_its() - 1].get_y())
            
                self.set_pieza(r[counter], i+1, self.get_jigsaw().get_tam() - self.get_its() - 1)
                i += 1
                counter = 0
            counter += 1
        # Delete RIGHT piezas:
        for i in r:
            self.get_random_piezas().pop(self.get_random_piezas().index(i))

        self.set_its(self.get_its()+1)

    def set_pieza(self, pieza, x, y):
        self.get_solve_jigsaw()[x][y] = pieza
        print('SET PIEZA : ', pieza, ' x ', pieza.get_x(), ' y ', pieza.get_y(), 'set: ', self.get_solve_jigsaw()[x][y])

    def solve_in_circles(self):
        its = self.get_its()
        j = self.get_jigsaw()
        # Find new edges:
        print('WHAT?')
        while self.check_solution() != True:
            print('1')
            self.set_solve_esquinas()
            print('2')
            self.set_solve_bordes()
            print('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA')
            for i, saw in enum(j):
                #print('jigsaw: ', i, ' - ', saw)
                sj = self.get_solve_jigsaw()
            for s in sj:
                print('hola')
                #print(s)
            #print(self.get_solve_jigsaw())

    def check_solution(self):
        return self.get_jigsaw() == self.get_solve_jigsaw()
        


In [546]:
j = Jigsaw('love', 8)

p = Player('ali-g', j)

p.solve_in_circles()

WHAT?
1
CORNERS: 
	 - downl rp <__main__.Ficha object at 0x0000025D39E43C50>  x  0  y  7
	 - upr rp <__main__.Ficha object at 0x0000025D393E5C10>  x  7  y  0
	 - downr rp <__main__.Ficha object at 0x0000025D39E40810>  x  7  y  7
	 - upl rp <__main__.Ficha object at 0x0000025D37B47050>  x  0  y  0
2
6 6 6 6
[<__main__.Ficha object at 0x0000025D393E6F50>, <__main__.Ficha object at 0x0000025D393E7690>, <__main__.Ficha object at 0x0000025D38D3DB10>, <__main__.Ficha object at 0x0000025D393E7910>, <__main__.Ficha object at 0x0000025D3A073ED0>, <__main__.Ficha object at 0x0000025D393E4CD0>]
[<__main__.Ficha object at 0x0000025D393E77D0>, <__main__.Ficha object at 0x0000025D3A21CD90>, <__main__.Ficha object at 0x0000025D39E40C10>, <__main__.Ficha object at 0x0000025D393E4610>, <__main__.Ficha object at 0x0000025D39E41510>, <__main__.Ficha object at 0x0000025D39E42010>]
[<__main__.Ficha object at 0x0000025D39E41D10>, <__main__.Ficha object at 0x0000025D39E43210>, <__main__.Ficha object at 0x000

IndexError: list index out of range

#### Exercise 7.7
**Chat Server:** Explain how you would design a chat server. In particular, provide details about the various backend components, classes, and methods. What would be the hardest problems to solve?

**Hints:** 
- #213: As always, scope the problem. Are "friendships" mutual? Do status messages exist? Do you support group chat?
- #245: This is a good problem to think about the major system components or technologies that would be useful.
- #271: How will you know if a user signs offiine? 

In [1]:
from enum import Enum

In [56]:
class Status(Enum):
    ONLINE = 0
    OFFLINE = 1
    BUSSY = 2
    FAKEOFF = 3
    AFK = 4

class HWStatus(Enum):
    ON = 0
    OFF = 1
    
class User:
    def __init__(self, platform):
        self.platform = platform
        self.name = None
        self.password = None
        self.username = None
        self.foto = None
        self.description = None 
        self.status = Status.OFFLINE
        self.friends = [] # Has to be mutual
        self.friends_requests = []
        self.mic_status = HWStatus.ON
        self.cam_status = HWStatus.ON
        self.call = None
        
        self.create_user()
                    
    def create_user(self):
        while self.get_name() == None:
            i = input('Introduce your Name: ')
            conf = input(f"Are you sure you want your name to be: {i}? ['yes', 'no']").lower()
            if conf == 'yes':
                self.set_name(i)
        while self.get_password() == None:
            i = input('Introduce your Password: ')
            conf = input(f"Are you sure you want your password to be: {i}? ['yes', 'no']").lower()
            if conf == 'yes':
                self.set_password(i)
        while self.get_username() == None:
            i = input('Itroduce your Username: ')
            conf = input(f"Are you sure you want your username to be: {i}? ['yes', 'no']").lower()
            if conf == 'yes':
                self.set_username(i)
        while self.get_foto() == None:
            i = input("Upload your profile Foto: ['upload', 'pass']")
            if i != 'pass':
                conf = input(f"Are you sure you want to upload this profile picture: {i}? ['yes', 'no']").lower()
                if conf == 'yes':
                    self.set_foto(i)
            else:
                break
        while self.get_description() == None:
            i = input("Write your description: ")
            conf = input(f"Are you sure you want your descriotion to be: {i}?")
            if conf == 'yes':
                    self.set_description(i)

        print('Congratulations your user is completed')
        self.get_platform().add_user(self)

        '''
        Estaria bien un control de errores apra comprobar que no existe ya ese Usurname / Nombre / Password
        '''

    def edit_user(self):
        choice = input("what do you want to edit: ['Name', 'Password', 'Username', 'Foto', 'Description', 'return']: ").lower()
        while choice not in ['name', 'password', 'username', 'foto', 'description', 'return']:
            choice = input("what do you want to edit: ['Name', 'Password', 'Username', 'Foto', 'Description', 'return']: ").lower()
        if choice == 'return':
            return
        elif choice == 'name':
            self.set_name(input("New Name: "))
        elif choice == 'password':
            self.set_password(input("New Password: "))
        elif choice == 'username':
            self.set_username(input("New Username: "))
        elif choice == 'foto':
            self.set_foto(input("New Username: "))
        elif choice == 'description':
            self.set_description("New Description: ")

        
    def get_platform(self):
        return self.platform

    def get_mic_status(self):
        return self.mic_status

    def silence_mic(self):
        self.mic_status = HWStatus.OFF

    def desilence_mic(self):
        self.mic_status = HWStatus.ON

    def get_cam_status(self):
        return self.cam_status

    def silence_cam(self):
        self.mic_status = HWStatus.OFF

    def desilence_cam(self):
        self.mic_status = HWStatus.ON
        
    def set_platform(self, p):
        self.platform = p
        
    # Getter and setter for 'name'
    def get_name(self):
        return self.name

    def set_name(self, name):
        self.name = name

    # Getter and setter for 'password'
    def get_password(self):
        return self.password

    def set_password(self, password):
        self.password = password

    # Getter and setter for 'username'
    def get_username(self):
        return self.username

    def set_username(self, username):
        self.username = username

    # Getter and setter for 'foto'
    def get_foto(self):
        return self.foto

    def set_foto(self, foto):
        self.foto = foto

    # Getter and setter for 'description'
    def get_description(self):
        return self.description

    def set_description(self, description):
        self.description = description

    # Getter and setter for 'status'
    def get_status(self):
        return self.status

    def set_status(self, status):
        self.status = status

    # Getter and setter for 'friends'
    def get_friends(self):
        return self.friends

    
    def add_friend(self, friend):
        self.get_friends().append(friend)
        friend.get_friends().append(self)
        
    '''
    ---------------------------------------------------------------------------------------------------
    '''
    def remove_friend(self, friend):
        self.get_friends().remove(friend)
        friend.get_friends().remove(self)
    '''
    ---------------------------------------------------------------------------------------------------
    '''

    def get_friends_request(self):
        return self.friends_requests

    def add_friends_requests(self, friend):
        if friend in self.get_friends():
            print('You are already friends, so we cant send the invitation!')
            
        elif friend in self.get_friends_request():
            print(f'{friend.get_name()} already invited you to be friends... You both are friends Now!!')
            self.add_friend(friend)
            self.get_friends_request().remove(friend)
        else:
            print(f'Invitation sended to: {friend.get_name()}')
            friend.get_friends_request().append(self)
        return

    '''
    Hay que hacer que si U1 manda invitación a U2:
    ----- U2 no puede enviar invitación a U1
        -------- O al acerlo directamente le acepta ----- ESTA ES LA MAS RAZONABLE 
        -------- O le envía un mensaje de que tiene una petición suya
    '''
    def accept_friends_requests(self):

        
        if self.get_friends_request() == []:
            print('No Friends Requests Right Now')
            return
        else:
            for f in self.get_friends():
                for rf in self.get_friends_request():
                    if f == rf:
                        print('already Friend With: ', f.get_name())
                        self.get_friends_request().remove(f)
                        
            for f in self.get_friends_request():
                f.print_user()
                choice = input(f"Do you want to accept {f.get_username()} as your friend? ['yes', 'no']..")
                while choice not in ['yes', 'no']:
                    choice = input(f"UPS.. Something whent Wrong... -- Do you want to accept {f.get_username()} as your friend? ['yes', 'no']..")
                if choice == 'yes':
                    self.add_friend(f)
                elif choice == 'no':
                    self.get_friends_requests().remove(f)
                    

    def print_user(self):
        print('USER: ', self.get_foto(), ' -- STATUS: ', self.get_status(), '\n\t - Name:', self.get_name(), ' -- \t Password: ', self.get_password(), '\n\t - Username: ', self.get_username(), '\n\t - Description: ', self.get_description())
        print()
        
    def print_friends(self):
        print('FRIENDS OF: ', self.get_username())
        if self.friends == []:
            print('You dont have any actual Friend, go add some...\n')

        else:
            for i, f in enumerate(self.get_friends()):
                print(f"\n - {i}: {f}")
            print()

    def get_call(self):
        return self.call

    def set_call(self, call):
        self.call = call
        
    def answear_call(self, call):
        choice = input(f"{self.get_username} they are calling you. Do you wanna Join? ['yes', 'no']...")
        while choice not in ['yes', 'no']:
            choice = input(f"{self.get_username} they are calling you. Do you wanna Join? ['yes', 'no']...")
        if choice == 'yes':
            self.set_call(call)
            print('Asnwear')
            return 'yes'
        else:
            return 'no'

    def talk(self):
        if self.call == None:
            print('You are not in any group to talk')
        elif self.mic_status == HWStatus.OFF:
            print('Your MIC is silenced')
        else:
            self.call.user_speaks(self, input('Write a message... : '))



In [68]:
class Platform:
    def __init__(self, name):
        self.name = name
        self.users = []

    def add_user(self, user):
        self.users.append(user)

    def get_users(self):
        return self.users

    # Se puede hacer login
    def get_user(self, name):
        names = [u.get_name() for u in self.get_users()]
        if name in names:
            return self.get_users()[names.index(name)]
        print('That name dosnt exist or isnt registered')
        return False

    def login(self, name, password):
        user = self.get_user(name)
        if user.get_password() == password and user.get_status() == Status.OFFLINE:
            print('Succesfull Login')
            user.set_status(Status.ONLINE)

        else:
            print('Wrong Password, cant make login')

    def logout(self, user):
        user.set_status(Status.OFFLINE)
        print('GOODBYE')
        
    def print_users(self):
        if self.users == []:
            print('No users registered')
        else:
            for i, u in enumerate(self.get_users()):
                print(f"User {i}: ", u.print_user()) 

    def make_call(self, host, reciever):
        if host == None or reciever == None:
            raise Exception('Cant make Call with nobody')
        call = Call(host)
        whats_up = call.add_user(reciever)
        if whats_up == 0:
            return None
        else:
            return call

class Call:
    def __init__(self,host):
        self.users = []
        self.host = host
        self.reciever = []
        self.convo = []

    def add_user(self, new):
        if new.get_status == Status.OFFLINE:
            print('user not connected')
            return 0
        answear = new.answear_call(self)
        if answear == 'yes':
            self.reciever.append(new)
            self.host.set_call(self)
            return 1
        else:
            print(f"User {new.get_username()} declined the call")
            return 0

    def get_host(self):
        return self.host

    def get_recievers(self):
        return self.recievers

    def user_speaks(self, user, message):
        if message != '':
            self.convo.append(user.get_username()+ ' Wrote: '+ message)
            print(user.get_username(), ' Wrote: ', message)

    def view_all_conversation(self):
        for m in self.convo:
            print(m)
        

-----------
-----------

In [69]:
# Create Platform
p = Platform('dc')

In [70]:
# Create User
u = User(p)

Introduce your Name:  Guille
Are you sure you want your name to be: Guille? ['yes', 'no'] yes
Introduce your Password:  Alucard
Are you sure you want your password to be: Alucard? ['yes', 'no'] yes
Itroduce your Username:  Gojo
Are you sure you want your username to be: Gojo? ['yes', 'no'] yes
Upload your profile Foto: ['upload', 'pass'] upload
Are you sure you want to upload this profile picture: upload? ['yes', 'no'] yes
Write your description:  I am the human being with the greates will power and potential ever born on earth. As so, I am going to use it to achieve all the blessings that await me, one by one, living rekless and free, being myself
Are you sure you want your descriotion to be: I am the human being with the greates will power and potential ever born on earth. As so, I am going to use it to achieve all the blessings that await me, one by one, living rekless and free, being myself? yes


Congratulations your user is completed


In [71]:
# Print User 
u.print_user()
u.print_friends()

USER:  upload  -- STATUS:  Status.OFFLINE 
	 - Name: Guille  -- 	 Password:  Alucard 
	 - Username:  Gojo 
	 - Description:  I am the human being with the greates will power and potential ever born on earth. As so, I am going to use it to achieve all the blessings that await me, one by one, living rekless and free, being myself

FRIENDS OF:  Gojo
You dont have any actual Friend, go add some...



In [72]:
u2 = User(p)
u3 = User(p)
u4 = User(p)

Introduce your Name:  Zero
Are you sure you want your name to be: Zero? ['yes', 'no'] yes
Introduce your Password:  Zero
Are you sure you want your password to be: Zero? ['yes', 'no'] yes
Itroduce your Username:  Zero
Are you sure you want your username to be: Zero? ['yes', 'no'] yes
Upload your profile Foto: ['upload', 'pass'] upload
Are you sure you want to upload this profile picture: upload? ['yes', 'no'] yes
Write your description:  Zero
Are you sure you want your descriotion to be: Zero? yes


Congratulations your user is completed


Introduce your Name:  8
Are you sure you want your name to be: 8? ['yes', 'no'] yes
Introduce your Password:  8
Are you sure you want your password to be: 8? ['yes', 'no'] yes
Itroduce your Username:  8
Are you sure you want your username to be: 8? ['yes', 'no'] yes
Upload your profile Foto: ['upload', 'pass'] pass
Write your description:  yes
Are you sure you want your descriotion to be: yes? yes


Congratulations your user is completed


Introduce your Name:  XXX
Are you sure you want your name to be: XXX? ['yes', 'no'] yes
Introduce your Password:  XXX
Are you sure you want your password to be: XXX? ['yes', 'no'] yes
Itroduce your Username:  XXX
Are you sure you want your username to be: XXX? ['yes', 'no'] yes
Upload your profile Foto: ['upload', 'pass'] pass
Write your description:  XXX
Are you sure you want your descriotion to be: XXX? yes


Congratulations your user is completed


In [73]:
# Print Platform Users
p.print_users()

USER:  upload  -- STATUS:  Status.OFFLINE 
	 - Name: Guille  -- 	 Password:  Alucard 
	 - Username:  Gojo 
	 - Description:  I am the human being with the greates will power and potential ever born on earth. As so, I am going to use it to achieve all the blessings that await me, one by one, living rekless and free, being myself

User 0:  None
USER:  upload  -- STATUS:  Status.OFFLINE 
	 - Name: Zero  -- 	 Password:  Zero 
	 - Username:  Zero 
	 - Description:  Zero

User 1:  None
USER:  None  -- STATUS:  Status.OFFLINE 
	 - Name: 8  -- 	 Password:  8 
	 - Username:  8 
	 - Description:  yes

User 2:  None
USER:  None  -- STATUS:  Status.OFFLINE 
	 - Name: XXX  -- 	 Password:  XXX 
	 - Username:  XXX 
	 - Description:  XXX

User 3:  None


In [74]:
# Get specific user // LOG IN and LOG OUT??
guille = p.get_user('Guille')

print('status', guille.get_status())
p.login(guille.get_name(), 'Alucard')
print('status', guille.get_status())
p.logout(guille)
print('status', guille.get_status())

# Edit User
guille.edit_user()

status Status.OFFLINE
Succesfull Login
status Status.ONLINE
GOODBYE
status Status.OFFLINE


what do you want to edit: ['Name', 'Password', 'Username', 'Foto', 'Description', 'return']:  Return


In [75]:
guille.print_user()

USER:  upload  -- STATUS:  Status.OFFLINE 
	 - Name: Guille  -- 	 Password:  Alucard 
	 - Username:  Gojo 
	 - Description:  I am the human being with the greates will power and potential ever born on earth. As so, I am going to use it to achieve all the blessings that await me, one by one, living rekless and free, being myself



In [76]:
# Users:
z = p.get_user('Zero')
eight = p.get_user('8')
x = p.get_user('XXX')

# 1 ask for Friends
guille.add_friends_requests(z)
# ---- Z has to accept
z.accept_friends_requests()
# ----- RESULT

# Both ask for friends
guille.add_friends_requests(eight)
eight.add_friends_requests(guille)

# What if we already friends?
guille.add_friends_requests(eight)
eight.add_friends_requests(guille)

# Add last
guille.add_friends_requests(x)
x.add_friends_requests(guille)

Invitation sended to: Zero
USER:  upload  -- STATUS:  Status.OFFLINE 
	 - Name: Guille  -- 	 Password:  Alucard 
	 - Username:  Gojo 
	 - Description:  I am the human being with the greates will power and potential ever born on earth. As so, I am going to use it to achieve all the blessings that await me, one by one, living rekless and free, being myself



Do you want to accept Gojo as your friend? ['yes', 'no'].. yes


Invitation sended to: 8
Guille already invited you to be friends... You both are friends Now!!
You are already friends, so we cant send the invitation!
You are already friends, so we cant send the invitation!
Invitation sended to: XXX
Guille already invited you to be friends... You both are friends Now!!


In [77]:
for f in guille.get_friends():
    print(f.name)

print()
for f in z.get_friends():
    print(f.name)
    
print()
for f in eight.get_friends():
    print(f.name)

Zero
8
XXX

Guille

Guille


In [78]:
# Make Calls // MESAGES
p.make_call(guille, eight)

# Close Call

# Make Call -- Add people to call, expell people from call

# Make mic silenced - make mic unsilenced

# Make cam on - off

# Close call

<bound method User.get_username of <__main__.User object at 0x000001D83B000750>> they are calling you. Do you wanna Join? ['yes', 'no']... yes


Asnwear


In [79]:
guille.get_call()

In [80]:
eight.get_call()

In [81]:
guille.talk()

Write a message... :  Bro lo voyyyy a lograr


Gojo  Wrote:  Bro lo voyyyy a lograr


In [82]:
eight.talk()

Write a message... :  Eres la polla bro


8  Wrote:  Eres la polla bro


In [84]:
guille.get_call().view_all_conversation()

Gojo Wrote: Bro lo voyyyy a lograr
8 Wrote: Eres la polla bro


--------------------------------
----------------------------
------------------------------
----------------------------
----------------------------
----------------------------


In [ ]:
from enum import Enum

class Status(Enum):
    ONLINE = 0
    OFFLINE = 1
    BUSY = 2
    FAKEOFF = 3
    AFK = 4

class HWStatus(Enum):
    ON = 0
    OFF = 1

class User:
    def __init__(self, platform, name, password, username, foto=None, description=None):
        self.platform = platform
        self.name = name
        self.password = password
        self.username = username
        self.foto = foto
        self.description = description
        self.status = Status.OFFLINE
        self.friends = set()  # Use a set for friends to ensure uniqueness
        self.friends_requests = set()  # Use a set for friend requests
        self.mic_status = HWStatus.ON
        self.cam_status = HWStatus.ON
        self.call = None

        self.platform.add_user(self)

    # Add getter and setter methods for attributes here

    def edit_user(self):
        # Implement editing user details

    def add_friend(self, friend):
        if friend != self:
            self.friends.add(friend)
            friend.friends.add(self)

    def remove_friend(self, friend):
        if friend in self.friends:
            self.friends.remove(friend)
            friend.friends.remove(self)

    def add_friends_requests(self, friend):
        if friend != self:
            if friend not in self.friends and friend not in self.friends_requests:
                self.friends_requests.add(friend)

    def accept_friends_requests(self):
        new_friends = [friend for friend in self.friends_requests if friend not in self.friends]
        for friend in new_friends:
            print(f'Accepting friend request from {friend.username}')
            self.add_friend(friend)
        self.friends_requests.difference_update(new_friends)

    # Other methods and properties

class Platform:
    def __init__(self, name):
        self.name = name
        self.users = set()  # Use a set for users

    def add_user(self, user):
        self.users.add(user)

    # Implement other methods for managing users

class Call:
    def __init__(self, host):
        self.users = set()
        self.host = host
        self.convo = []

    def add_user(self, new):
        if new.status == Status.OFFLINE:
            print('User not connected')
            return False
        elif new == self.host:
            print('Cannot add the host as a participant')
            return False
        elif new in self.users:
            print(f'User {new.username} is already part of this call')
            return False
        else:
            self.users.add(new)
            return True

    # Implement other methods for managing the call

    def user_speaks(self, user, message):
        if message:
            self.convo.append(f'{user.username} wrote: {message}')
            print(f'{user.username} wrote: {message}')

    # Other methods

# Implementation of the Platform class and other missing parts

#### Exercise 7.8
**Othello:** Othello is played as follows: Each Othello piece is white on one side and black on the other. When a piece is surrounded by its opponents on both the left and right sides, or both the top and bottom, it is said to be captured and its color is flipped. On your turn, you must capture at least one of your opponent's pieces. The game ends when either user has no more valid moves. The win is assigned to the person with the most pieces. Implement the object-oriented design for Othello.

**Hints:**
- #179: Should white pieces and black pieces be the same class? What are the pros and cons of this?
- #228: What class should maintain the score?

In [143]:
from enum import Enum
import random

In [121]:
class Color(Enum):
    WHITE = 0
    BLACK = 1

class Piece:
    def __init__(self, color):
        self.color = color

    def get_color(self):
        return self.color

    def flip(self):
        if self.color == Color.WHITE:
            self.color = Color.BLACK
        else:
            self.color = Color.WHITE

class Player:
    def __init__(self, name):
        self.name = name
        self.color = None

    def get_name(self):
        return self.name

    def get_color(self):
        return self.color

    def set_color(self, color):
        self.color = color


class Board:
    def __init__(self):
        self.board = [[None for _ in range(8)] for _ in range(8)]
        self.n_whites = 0
        self.n_blacks = 0

    def get_board(self):
        return self.board

    def get_n_whites(self):
        return self.n_whites

    def get_n_blacks(self):
        return self.n_blacks

    def score(self):
        white, black = 0, 0
        
        for y in self.get_board():
            for x in y:
                if x != None:
                    if x.get_color() == Color.WHITE: 
                        white += 1
                    else:
                        black += 1
                        
        self.n_whites = white
        self.n_blacks = black

    def start_board(self):
        self.get_board()[3][3] = Piece(Color.WHITE)
        self.get_board()[3][4] = Piece(Color.BLACK)
        self.get_board()[4][3] = Piece(Color.BLACK)
        self.get_board()[4][4] = Piece(Color.WHITE)
        self.score()
        print(self)

    def start_board_as_challenge(self):
        self.get_board()[0][1] = Piece(Color.BLACK)
        self.get_board()[0][2] = Piece(Color.BLACK)
        self.get_board()[0][3] = Piece(Color.WHITE)
        self.get_board()[0][4] = Piece(Color.WHITE)
        self.get_board()[0][5] = Piece(Color.BLACK)

        self.get_board()[1][1] = Piece(Color.WHITE)
        self.get_board()[1][2] = Piece(Color.WHITE)
        self.get_board()[1][3] = Piece(Color.WHITE)
        self.get_board()[1][4] = Piece(Color.WHITE)
        self.get_board()[1][5] = Piece(Color.WHITE)

        self.get_board()[2][1] = Piece(Color.WHITE)
        self.get_board()[2][2] = Piece(Color.WHITE)
        self.get_board()[2][3] = Piece(Color.WHITE)
        self.get_board()[2][4] = Piece(Color.WHITE)
        self.get_board()[2][5] = Piece(Color.WHITE)

        self.get_board()[3][1] = Piece(Color.BLACK)
        self.get_board()[3][3] = Piece(Color.WHITE)
        self.get_board()[3][4] = Piece(Color.WHITE)
        self.get_board()[3][5] = Piece(Color.WHITE)
        self.get_board()[3][6] = Piece(Color.BLACK)
        
        self.score()
        print(self)
        
    def __str__(self):
        board = self.get_board()
        board_str = "   0 1 2 3 4 5 6 7 \n"
        board_str += "_____________________\n"
        i = 0
        for i,row in enumerate(board):
            board_str += f"{i}|"
            for coord in row:
                if coord == None:
                    board_str += " _"
                else:
                    if coord.get_color() == Color.BLACK:
                        board_str += " b"
                    else:
                        board_str += " w"
            if i == 2 or i == 4 or i == 5:
                if i == 2:
                    board_str += f" |        Scores: \n"
                elif i == 4:
                    board_str += f" |        whites: {self.get_n_whites()}\n"
                else:
                    board_str += f" |        blacks: {self.get_n_blacks()}\n"
            else:
                board_str += " |\n"
                
        board_str += "_____________________"
        
        return board_str

    def valid_move(self, color, x, y):
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, 1), (1, -1), (-1, -1)]
        
        if self.get_board()[y][x] != None:
            print('Sitio ya cogido, NO VALIDO')
            
        for dx, dy in directions:
            nx, ny = x+dx, y+dy
            flag = False
            while 0 <= nx < 8 and 0 <= ny < 8:
                if self.get_board()[ny][nx] == None:
                    break
                elif self.get_board()[ny][nx].get_color() == color:
                    if flag == True:
                        return True
                    break
                else:
                    flag = True
                nx += dx
                ny += dy

        return False
            
    def put_piece(self, color, x = None, y = None, rec = 0):
        i = rec
        if x is None and y is None:
            x = int(input('Select X coordinate: '))
            y = int(input('Select Y coordinate: '))

            while not self.valid_coord(color, x, y):
                x = int(input('Select X coordinate AGAIN: '))
                y = int(input('Select Y coordinate AGAIN: '))

        print(f"('x:{x}' - 'y:{y}')")
        # Se supone que el movimiento es 100% Válido:
        if rec == 0:
            self.get_board()[y][x] = Piece(color)

        directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, 1), (1, -1), (-1, -1)]
        flips_dirs = []
        
        for dx, dy in directions:
            nx,ny = x+dx, y+dy
            flips = []
            
            while 0 <= nx < 8 and 0 <= ny < 8:
                if self.get_board()[ny][nx] == None:
                    flips = []
                    break
                elif self.get_board()[ny][nx].get_color() == color:
                    if flips != []:
                        flips_dirs.append(flips)
                    break
                else:
                    flips.append((nx, ny))
                    
                nx += dx
                ny += dy

        # Flip the corresponding ones
        if flips_dirs != []:
            print('flip_dirs: ', flips_dirs)
            for flips in flips_dirs:
                for piece in flips:
                    self.get_board()[piece[1]][piece[0]].flip()

        # Count Score
        self.score()

        # Print Board
        print(self)
        
        # Recursivity over the recent flips
        i += 1
        print('Rec Nº: ', i)
        if flips_dirs != []:
            for flips in flips_dirs:
                for piece in flips:
                    self.put_piece(color, piece[0], piece[1], i)

        

    
class Game:
    def __init__(self, player1, player2):
        self.name = 'Othello'
        self.player1 = Player(player1)
        self.player2 = Player(player2)
        self.board = Board()
        self.turn = None

    def get_p1(self):
        return self.player1
    def get_p2(self):
        return self.player2

    def get_board(self):
        return self.board

    def get_turn(self):
        return self.turn

    def set_turn(self, turn):
        self.turn = turn
        
    def next_turn(self):
        if self.get_turn() == None:
            if self.get_p1().color == Color.BLACK:
                self.turn = self.get_p1()
            else:
                self.turn = self.get_p2()
        elif self.get_turn() == self.get_p1():
            self.set_turn(self.get_p2())
        else:
            self.set_turn(self.get_p1())

    def start_game(self, board = 0):
        print('START GAME!!! \n')
        
        # Who Starts ? 
        self.select_turns()
        # Start Board 
        if board == 0:
            self.get_board().start_board()
        else:
            self.get_board().start_board_as_challenge()

        # until the end:
        while self.end_condition() == False: 
            # Player Selects Coord to place New Piece
            x = int(input('Select X coordinate: '))
            y = int(input('Select Y coordinate: '))
            print('vale')
            while not self.get_board().valid_move(self.get_turn().get_color(), x, y):
                x = int(input('Select X coordinate AGAIN: '))
                y = int(input('Select Y coordinate AGAIN: '))

            print('vale')
            
            # Move:
            self.get_board().put_piece(self.get_turn().get_color(), x, y)

            # Next Turn:
            self.next_turn()
            print('\nLEGO: ', self.get_turn().get_name())
    
    
    def end_condition(self):
        p = self.get_turn()
        
        if self.get_board().get_n_whites() + self.get_board().get_n_blacks() == 64:
            print('All pieces Done')
            return True

        else:
            '''if p == self.get_p1():
                if self.get_board().get_possible_moves(p.get_color()) == False and self.get_board().get_possible_moves(self.get_p2().get_color()) == False:
                    print('Players cant move')
                    return True
            elif p == self.get_p2():
                if self.get_board().get_possible_moves(p.get_color()) == False and self.get_board().get_possible_moves(self.get_p1().get_color()) == False:
                    print('Players cant move')
                    return True'''
        
        print('False')
        return False
        
    
    def select_turns(self):
        choice = input("Choose option: ['1','2','3'] \n\t 1.- P1 Choose Whites \n\t 2.- P2 Choose Whites \n\t 3.- Choose Sides Randomly")
        while choice not in ['1', '2', '3']:
                choice = input("Choose option: ['1','2','3'] \n\t 1.- Choose side P1 \n\t 2.- Choose side P2 \n\t 3.- Choose Sides Randomly")

        if choice == '1':
            print(f'P1 {self.get_p1().get_name()} Whites')
            self.get_p1().set_color(Color.WHITE)
            self.get_p2().set_color(Color.BLACK)
            self.next_turn()
            print(f' Black Starts: P2 {self.get_turn().get_name()} Turn')
        elif choice == '2':
            print(f'P2 {self.get_p2().get_name()} Whites')
            self.get_p1().set_color(Color.BLACK)
            self.get_p2().set_color(Color.WHITE)
            self.next_turn()
            print(f' Black Starts: P1 {self.get_turn().get_name()} Turn')
        elif choice == '3':
            print('Choose Sides by Flipping Coin: ')
            rand_n = random.randint(1,2)
            if rand_n == 1:
                print(f'P1 {self.get_p1().get_name()} Whites')
                self.get_p1().set_color(Color.WHITE)
                self.get_p2().set_color(Color.BLACK)
                self.next_turn()
                print(f' Black Starts: P2 {self.get_turn().get_name()} Turn')
            else:
                print(f'P2 {self.get_p2().get_name()} Whites')
                self.get_p1().set_color(Color.BLACK)
                self.get_p2().set_color(Color.WHITE)
                self.next_turn()
                print(f' Black Starts: P1 {self.get_turn().get_name()} Turn')
            

In [122]:
p1 = Player('Gojo')
p2 = Player('Itadori')
g = Game(p1,p2)
g.start_game(0)

START GAME!!! 



Choose option: ['1','2','3'] 
	 1.- P1 Choose Whites 
	 2.- P2 Choose Whites 
	 3.- Choose Sides Randomly 1


P1 <__main__.Player object at 0x00000226AFD112D0> Whites
 Black Starts: P2 <__main__.Player object at 0x00000226AFCF7E90> Turn
   0 1 2 3 4 5 6 7 
_____________________
0| _ _ _ _ _ _ _ _ |
1| _ _ _ _ _ _ _ _ |
2| _ _ _ _ _ _ _ _ |        Scores: 
3| _ _ _ w b _ _ _ |
4| _ _ _ b w _ _ _ |        whites: 2
5| _ _ _ _ _ _ _ _ |        blacks: 2
6| _ _ _ _ _ _ _ _ |
7| _ _ _ _ _ _ _ _ |
_____________________
False


Select X coordinate:  2
Select Y coordinate:  c


ValueError: invalid literal for int() with base 10: 'c'

#### Exercise 7.9
**Circular Array:** Implement a CircularArray class that supports an array-like data structure which can be efficiently rotated. If possible, the class should use a generic type (also called a template), and should support iteration via the standard f or (Obj o : circularArray) notation.

**Hints:**
- #389: The rotate () method should be able to run in 0(1) time.

In [56]:
class CircularArray():
    def __init__(self):
        self.array = []
        self.head = 0

    def add_element(self, data):
        self.array.append(data)
        
    def get_head(self):
        return self.head
        
    def rotate(self):
        self.head += 1

        print('head = ', self.head, 'len = ', len(self.array))
        
        for i in range(len(self.array)):
            print(self.array[(self.head+i)%len(self.array)])

a = CircularArray()

a.add_element(0)
a.add_element(1)
a.add_element(2)
a.add_element(3)
a.add_element(4)
a.add_element(5)
a.add_element(6)

print(a.head)

print(a.array)
a.rotate()
a.rotate()
a.rotate()
a.rotate()
a.rotate()
a.rotate()
a.rotate()
        

0
[0, 1, 2, 3, 4, 5, 6]
head =  1 len =  7
1
2
3
4
5
6
0
head =  2 len =  7
2
3
4
5
6
0
1
head =  3 len =  7
3
4
5
6
0
1
2
head =  4 len =  7
4
5
6
0
1
2
3
head =  5 len =  7
5
6
0
1
2
3
4
head =  6 len =  7
6
0
1
2
3
4
5
head =  7 len =  7
0
1
2
3
4
5
6


In [58]:
class CircularArray:
    def __init__(self, capacity):
        self.capacity = capacity
        self.array = [None] * capacity
        self.head = 0
        self.size = 0

    def add_element(self, data):
        if self.size < self.capacity:
            self.array[(self.head + self.size) % self.capacity] = data
            self.size += 1
        else:
            print('Handle the case where the array is full or exceeded capacity.')

    def get_head(self):
        return self.head

    def rotate(self):
        if self.size > 0:
            self.head = (self.head + 1) % self.capacity

    def __iter__(self):
        for i in range(self.size):
            yield self.array[(self.head + i) % self.capacity]

#### Exercise 7.10
**Minesweeper:** Design and implement a text-based Minesweeper game. Minesweeper is the classic single-player computer game where an NxN grid has B mines (or bombs) hidden across the grid. The remaining cells are either blank or have a number behind them. The numbers reflect the number of bombs in the surrounding eight cells. The user then uncovers a cell. If it is a bomb, the player loses. If it is a number, the number is exposed. If it is a blank cell, this cell and all adjacent blank cells (up to and including the surrounding numeric cells) are exposed. The player wins when all non-bomb cells are exposed. The player can also flag certain places as potential bombs. This doesn't affect game play, other than to block the user from accidentally clicking a cell that is thought to have a bomb. (Tip for the reader: if you're not familiar with this game, please play a few rounds on line first.) 

**Hints:**
- #351: Should number cells, blank cells, and bomb cells be separate classes?
- #361: What is the algorithm to place the bombs around the board? 
- #377: To place the bombs randomly on the board:Think about the algorithm to shuffle a deck 
of cards. Can you apply a similar technique? 
- #386 How do you count the number of bombs neighboring a cell? Will you iterate through all 
cells?:
- #39 When you click on a blank cell, what is the algorithm to expand the neighboring cells? 9:

In [117]:
from enum import Enum
import random

class Game:
    def __init__(self):
        self.board = None

    def ini_game(self, size):
        #size = int(input('select size of the game: NxN size board and N bombs'))
        self.set_board(Board(size))
        print(self.get_board())

    def get_board(self):
        return self.board

    def set_board(self, b):
        self.board = b

    def get_finished(self):
        return finished

    def set_finished(self, t):
        self.finished = t

    def start_game(self, size):
        self.ini_game(size)
        flag_counter = 0
        flag_coords = []
        
        while self.get_board().get_end_game() != True:
            
            action = input("Seleect Action: ['Flag', 'Open Cell']")
            while action not in ['Flag', 'Open Cell']:
                print('Something whent wrong: ')
                action = input("Seleect Action: ['Flag', 'Open Cell']")

            print('Select Coordenates [x = Col,  y = Row]')
            x = int(input('Select the X coordenate you want to open next'))
            y = int(input('Select the Y coordenate you want to open next'))
            
            while 0 >= x > self.get_board().get_size()-1 or 0 >= y > self.get_board().get_size()-1:
                print('Something whent wrong: ')
                x = int(input('Select the X coordenate you want to open next'))
                y = int(input('Select the Y coordenate you want to open next'))

            if action == 'Open Cell':
                self.get_board().open_piece(x, y)

            else:
                self.get_board().flag_piece(x, y)
                flag_coords.append((x,y))
                flag_counter += 1
                
            print(self.get_board())

            checker = 10
            if flag_counter == self.get_board().get_size():
                for col, row in flag_coords:
                    print(f'col = {col}, row = {row}')
                    if self.get_board().get_grid()[row][col].get_type() != PieceType.BOMB:
                        print('Some Flags Are wrong')
                        print(f'x = {x}, y = {y}, is type --> {self.get_board().get_grid()[row][col].get_type()} ')

                    else:
                        checker += 1

                if flag_counter == checker:
                    print('YOU WON THE GAME !!!!')
                    self.get_board().set_end_game(True)
                        
        

class Board:
        
    def __init__(self, size):
        self.size = size
        self.grid = [[Piece() for i in range(size)] for j in range(size)]
        print(f'grid_size = {len(self.grid)}, {len(self.grid[0])}')
        self.ramdomize()
        
        self.end_game = False
        
    
    def ramdomize(self):
        indexs = list(range(100))
        random.shuffle(indexs)

        for i in range(self.size):
            print(f'CREATING BOMBS: {i}')
            index = indexs[i]
            row, col = divmod(index, self.size)
            print(f'x = {col}, y = {row}')
            self.grid[row][col].transform_to_bomb()
            coords = self.get_8(row, col)
            for r, c in coords:
                self.grid[r][c].transform_to_number()
            self.flag_piece(col, row)
            
    def get_8(self, row, col):
        coords = [(row-1, col-1), (row-1, col), (row-1, col+1), (row, col-1), (row, col+1), (row+1, col-1), (row+1, col), (row+1, col+1)]
        #is_border??
        if row == 0:
            'Supperior Extreme -- Cant substract to row'
            remover = row-1
            coords = [(r,c) for r, c in coords if r != remover]

        if col == 0: 
            'left Extreme -- Cant substract to col'
            remover = col-1
            coords = [(r,c) for r, c in coords if c != remover]

        if row == self.size - 1:
            'Inferior Extreme -- Cant Add to Row'
            remover = self.size
            coords = [(r,c) for r, c in coords if r != remover]
            
        if col == self.size - 1:
            'Right Extreme -- Cant add to Col'
            remover = self.size
            coords = [(r,c) for r, c in coords if c != remover]

        return coords


    def __str__(self):
        board = self.get_grid()
        board_str = "    "
        for i in range(self.size):
            board_str += f"{i} "

        board_str += "\n_________________________\n"
        for i, row in enumerate(board):
            board_str += f"{i} :"
            for coord in row:
                if coord.flag == True:
                    board_str += " f"
                    
                elif coord.is_exposed == False:
                    board_str += " _"
                    
                if coord.flag == False and coord.is_exposed == True:
                    if type(coord.data) == int:
                        board_str += f" {coord.data}"
                    else:
                        board_str += " " + coord.data

            board_str += "  |\n"
        
        board_str += "_________________________\n"
        
        return board_str
    
    def get_grid(self):
        return self.grid

    def get_size(self):
        return self.size

    def set_size(self, value):
        if value >= 2:  # Adjust this condition as needed for your specific case
            self.size = value
        else:
            raise ValueError("Size must be at least 2")

    #def uncover_area(self, coords):
    def open_piece(self, x, y):
        if self.get_grid()[y][x].get_is_exposed() == False:
             # Expose Content on the Cell
            self.get_grid()[y][x].expose()
            
            # White Piece -- Recursiveness oppenning 8, unless its already oppened
            if(self.get_grid()[y][x].get_data() == None):
                print('White')
                self.get_grid()[y][x].set_data('x')
                coords = self.get_8(y, x)
                for row, col in coords:
                    self.open_piece(col, row)

            # Number
            elif(type(self.get_grid()[y][x].get_data()) == int):
                # Expose Content on the Cell
                self.get_grid()[y][x].expose()
                
        
            # Bomb
            else:
                # Expose Content on the Cell
                self.get_grid()[y][x].expose()
                self.set_end_game(True)
                print('YOU LOOSE THE GAME')
                return
                
        #else:
            #print(f'Piece x = {x}, y = {y} Already Exposed')
    def flag_piece(self, x, y):
        self.get_grid()[y][x].make_flag()

    def get_end_game(self):
        return self.end_game

    def set_end_game(self, t):
        self.end_game = t
    
        
class PieceType(Enum):
    BLANK = 0
    NUMBER = 1
    BOMB = 2
    
class Piece:
    def __init__(self, data = None, type = None):
        if data == None:
            self.data = None
            self.type = PieceType.BLANK
        else:
            self.data = data
            self.type = type
            
        self.is_exposed = False
        self.flag = False

    def make_flag(self):
        if self.flag == False:
            self.flag = True
        else:
            self.flag = False

    def get_is_exposed(self):
        return self.is_exposed
        
    def expose(self):
        self.is_exposed = True

    def get_data(self):
        return self.data
        
    def get_type(self):
        return self.type
        
    def set_data(self, data):
        self.data = data
        
    def set_type(self, type):
        self.type = type

    def transform_to_bomb(self):
        self.data = 'O'
        self.type = PieceType.BOMB

    def transform_to_number(self):
        if self.type == PieceType.NUMBER:
            self.data += 1
        else:
            self.data = 1
            self.type = PieceType.NUMBER
        
        

In [118]:
g = Game()
g.start_game(10)

grid_size = 10, 10
CREATING BOMBS: 0
x = 2, y = 5
CREATING BOMBS: 1
x = 5, y = 8
CREATING BOMBS: 2
x = 2, y = 0
CREATING BOMBS: 3
x = 1, y = 4
CREATING BOMBS: 4
x = 8, y = 4
CREATING BOMBS: 5
x = 8, y = 1
CREATING BOMBS: 6
x = 3, y = 1
CREATING BOMBS: 7
x = 5, y = 3
CREATING BOMBS: 8
x = 1, y = 7
CREATING BOMBS: 9
x = 7, y = 2
    0 1 2 3 4 5 6 7 8 9 
_________________________
0 : _ _ f _ _ _ _ _ _ _  |
1 : _ _ _ f _ _ _ _ f _  |
2 : _ _ _ _ _ _ _ f _ _  |
3 : _ _ _ _ _ f _ _ _ _  |
4 : _ f _ _ _ _ _ _ f _  |
5 : _ _ f _ _ _ _ _ _ _  |
6 : _ _ _ _ _ _ _ _ _ _  |
7 : _ f _ _ _ _ _ _ _ _  |
8 : _ _ _ _ _ f _ _ _ _  |
9 : _ _ _ _ _ _ _ _ _ _  |
_________________________



Seleect Action: ['Flag', 'Open Cell'] Open Cell


Select Coordenates [x = Col,  y = Row]


Select the X coordenate you want to open next 0
Select the Y coordenate you want to open next 0


White
White
White
White
    0 1 2 3 4 5 6 7 8 9 
_________________________
0 : x 1 f _ _ _ _ _ _ _  |
1 : x 1 2 f _ _ _ _ f _  |
2 : x x 1 _ _ _ _ f _ _  |
3 : 1 1 1 _ _ f _ _ _ _  |
4 : _ f _ _ _ _ _ _ f _  |
5 : _ _ f _ _ _ _ _ _ _  |
6 : _ _ _ _ _ _ _ _ _ _  |
7 : _ f _ _ _ _ _ _ _ _  |
8 : _ _ _ _ _ f _ _ _ _  |
9 : _ _ _ _ _ _ _ _ _ _  |
_________________________



...

Seleect Action: ['Flag', 'Open Cell'] Flag


Select Coordenates [x = Col,  y = Row]


Select the X coordenate you want to open next 0
Select the Y coordenate you want to open next 4


    0 1 2 3 4 5 6 7 8 9 
_________________________
0 : x 1 f _ _ _ _ _ _ _  |
1 : x 1 2 f _ _ _ _ f _  |
2 : x x 1 _ _ _ _ f _ _  |
3 : 1 1 1 _ _ f _ _ _ _  |
4 : _ f _ _ _ _ _ _ f _  |
5 : _ _ f _ _ _ _ _ _ _  |
6 : _ _ _ _ _ _ _ _ _ _  |
7 : _ f _ _ _ _ _ _ _ _  |
8 : _ _ _ _ _ f _ _ _ _  |
9 : _ _ _ _ _ _ _ _ _ _  |
_________________________



KeyboardInterrupt: Interrupted by user

#### Exercise 7.11
**File System:** Explain the data structures and algorithms that you would use to design an in-memory file system. Illustrate with an example in code where possible. 

**Hints:**
- #141: This is not as complicated as it sounds. Start by making a list of the key objects in the system, then think about how they interact. 
- #216: What is the relationship between files and directories? 

In [119]:
import time

class Entry:
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.created = time.time()
        self.last_updated = time.time()
        self.last_accessed = time.time()

    def delete(self):
        if self.parent is None:
            return False
        return self.parent.delete_entry(self)

    def size(self):
        pass

    def get_full_path(self):
        if self.parent is None:
            return self.name
        else:
            return self.parent.get_full_path() + '/' + self.name

    def get_creation_time(self):
        return self.created

    def get_last_updated_time(self):
        return self.last_updated

    def get_last_accessed_time(self):
        return self.last_accessed

    def change_name(self, new_name):
        self.name = new_name

    def get_name(self):
        return self.name

class File(Entry):
    def __init__(self, name, parent, size):
        super().__init__(name, parent)
        self.content = ""
        self.size = size

    def size(self):
        return self.size

    def get_contents(self):
        return self.content

    def set_contents(self, content):
        self.content = content

class Directory(Entry):
    def __init__(self, name, parent):
        super().__init__(name, parent)
        self.contents = []

    def size(self):
        size = 0
        for entry in self.contents:
            size += entry.size()
        return size

    def number_of_files(self):
        count = 0
        for entry in self.contents:
            if isinstance(entry, Directory):
                count += 1  # Directory counts as a file
                count += entry.number_of_files()
            elif isinstance(entry, File):
                count += 1
        return count

    def delete_entry(self, entry):
        return self.contents.remove(entry)

    def add_entry(self, entry):
        self.contents.append(entry)

    def get_contents(self):
        return self.contents

# Example usage:
if __name__ == "__main__":
    root = Directory("FileSystem", None)
    documents = Directory("Documents", root)
    pictures = Directory("Pictures", root)

    root.add_entry(documents)
    root.add_entry(pictures)

    example_file = File("example.txt", documents, 100)
    example_file.set_contents("This is an example file.")
    documents.add_entry(example_file)

    # Print file system structure
    print(f"Root directory: {root.get_full_path()}")
    print(f"Number of files in root directory: {root.number_of_files()}")
    print(f"Contents of root directory: {[entry.get_name() for entry in root.get_contents()]}")

Root directory: FileSystem
Number of files in root directory: 3
Contents of root directory: ['Documents', 'Pictures']


#### Exercise 7.12
**Hash Table:** Design and implement a hash table which uses chaining (linked lists) to handle collisions.

**Hints:** 
- #287: In order to handle collisions, the hash table should be an array of linked lists. 
- #307: Think carefully about what information the linked list node needs to contain.

In [120]:
class HashTable:
    def __init__(self, size):
        self.size = size
        self.buckets = [None] * size

    def put(self, key, value):
        index = self.hash_function(key)
        if self.buckets[index] is None:
            self.buckets[index] = LinkedList()
        self.buckets[index].append(key, value)

    def get(self, key):
        index = self.hash_function(key)
        if self.buckets[index] is not None:
            return self.buckets[index].find(key)
        return None

    def hash_function(self, key):
        return len(key) % self.size

class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, key, value):
        new_node = Node(key, value)
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.next is not None:
                current = current.next
            current.next = new_node

    def find(self, key):
        current = self.head
        while current is not None:
            if current.key == key:
                return current.value
            current = current.next
        return None

# Example usage:
if __name__ == "__main__":
    hash_table = HashTable(5)  # Create a hash table with 5 buckets

    hash_table.put("John", 90)
    hash_table.put("Bob", 85)  # Collides with "John"
    hash_table.put("Alice", 88)
    hash_table.put("Eve", 77)

    print(hash_table.get("John"))  # Output: 90
    print(hash_table.get("Bob"))   # Output: 85
    print(hash_table.get("Eve"))   # Output: 77

90
85
77


Additional Questions: Threads and Locks (#16.3)
    
Hints start on page 662. 